In [2]:
import sys
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/poc")
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/plugins")
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/dags")

#Import libs python
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import date

#Import libs internas
from utils import spark_utils_session as utils

from hooks.hdfs.hdfs_helper import HdfsHelper
from jobs.job_base_config import BaseETLJobClass

import poc_helper
poc_helper.load_env("PROD")

'.env_file' loaded!
ENV 'PROD' configured!


In [3]:
def get_session(profile: str, dynamic_allocation_enabled: bool = True) -> utils.DBASparkAppSession:
    """Generates DBASparkAppSession."""
    
    app_name = "tsevero_ecd_pc_ml"
    
    spark_builder = (utils.DBASparkAppSession
                     .builder
                     .setAppName(app_name)
                     .usingProcessProfile(profile)
                    )
    
    if dynamic_allocation_enabled:
        spark_builder.autoResourceManagement()

    return spark_builder.build()

session = get_session(profile='efd_t2')

2025-11-13T22:04:49.569112Z [info     ] Using json file settings.      [root] loc=spark_utils_session.py:301
2025-11-13T22:04:49.570997Z [info     ] Exporting default ENV.         [root] loc=spark_utils_session.py:305
2025-11-13T22:04:49.571483Z [info     ] Exporting custom ENVs.         [root] loc=spark_utils_session.py:338
2025-11-13T22:04:49.571894Z [info     ] Building profile 'efd_t2'.     [root] loc=spark_utils_session.py:221
2025-11-13T22:04:49.572355Z [info     ] Not enough info for building the kerberos client. Ignoring it [root] loc=spark_utils_session.py:284


:: loading settings :: url = jar:file:/opt/cloudera/parcels/SPARK3-3.5.4.3.5.7191000.0-30-1.p0.68499982/lib/spark3/jars/ivy-2.5.2.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/tsevero/.ivy2/cache
The jars for the packages stored in: /home/tsevero/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
com.oracle.database.security#oraclepki added as a dependency
com.oracle.database.security#osdt_core added as a dependency
com.oracle.database.security#osdt_cert added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-281f0f11-bd7a-4333-a048-ccd78fb0fe55;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.18.0 in central
	found commons-io#commons-io;2.11.0 in central
	found org.glassfish.jaxb#txw2;3.0.2 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.3.0 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 in central
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.9.2 in central
	found com.oracle.database.security#oraclepki;21.19.0.0 in central
	found com.oracle.databa

In [4]:
session.sparkSession.sql("SHOW DATABASES").show(truncate=False)

Hive Session ID = f3baefac-bad0-4d72-a757-9d3824767e61


+------------------+
|namespace         |
+------------------+
|anac              |
|bcadastro         |
|bpe               |
|c115              |
|ccc               |
|ccg               |
|cte               |
|default           |
|destda            |
|detran_share      |
|dime              |
|due               |
|efd               |
|fci               |
|gecob             |
|gescol            |
|gessimples        |
|gplam             |
|information_schema|
|malhas            |
+------------------+
only showing top 20 rows



In [5]:
# ============================================================================
# CONFIGURAÇÃO INICIAL
# ============================================================================

import sys
import warnings
from datetime import datetime, date
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# PySpark imports com aliases para evitar conflitos
from pyspark.sql.functions import (
    col as spark_col, 
    sum as spark_sum, 
    avg as spark_avg,
    count as spark_count,
    when as spark_when,
    desc as spark_desc,
    asc as spark_asc,
    round as spark_round,
    concat as spark_concat,
    lit as spark_lit,
    max as spark_max,
    min as spark_min,
    stddev as spark_stddev,
    countDistinct as spark_countDistinct
)
from pyspark.sql.types import DoubleType, IntegerType

# Configurações de visualização
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (16, 8)
plt.rcParams['font.size'] = 11

# ✅ CORREÇÃO: Não usar abs() que conflita com PySpark
# pd.set_option('display.float_format', lambda x: f'{x:,.2f}' if abs(x) > 0.01 else f'{x:.6f}')
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

# Acesso ao Spark
spark = session.sparkSession

print("=" * 80)
print("🔍 SISTEMA DE ANÁLISE")
print("=" * 80)
print(f"Sessão Spark: {spark.sparkContext.appName}")
print(f"Versão Spark: {spark.version}")
print(f"Iniciado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 80)

🔍 SISTEMA DE ANÁLISE
Sessão Spark: tsevero_ecd_pc_ml
Versão Spark: 3.5.4.3.5.7191000.0-30
Iniciado em: 2025-11-13 19:05:08


In [6]:
"""
================================================================================
PIPELINE COMPLETO DE CLASSIFICAÇÃO ECD - VERSÃO COMPLETA
================================================================================
Atualizado em: 2025-11-13

Este pipeline integra todas as funcionalidades do notebook original:
1. Ground Truth (identificação de empresas bem comportadas)
2. Feature Engineering (extração de características estruturais)
3. Classificação Estrutural (regras baseadas em padrões)
4. Classificação por Dicionários (palavras-chave)
5. Validação Contábil (equação, natureza, hierarquia)

Autor: Sistema de Classificação ECD - SEFAZ/SC
================================================================================
"""

from pyspark.sql.functions import (
    col, when, lit, regexp_extract, substring, upper, lower, 
    length, regexp_replace, count, sum as spark_sum, avg,
    desc, asc, row_number, dense_rank, countDistinct,
    coalesce, greatest, least, round as spark_round,
    concat, concat_ws, trim, array, array_contains,
    levenshtein, abs as spark_abs, max as spark_max, min as spark_min,
    stddev as spark_stddev
)
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType, IntegerType, StringType, ArrayType, StructType, StructField
from datetime import datetime
import sys

# ============================================================================
# SUPRIMIR WARNINGS DO JANINO
# ============================================================================

# Configurar logging do Spark para suprimir warnings do CodeGenerator
spark.sparkContext.setLogLevel("ERROR")

# Configurar propriedades do Spark para otimizar geração de código
spark.conf.set("spark.sql.codegen.wholeStage", "false")  # Desabilitar whole-stage codegen
spark.conf.set("spark.sql.codegen.maxFields", "50")     # Reduzir máximo de campos

print("✅ Configurações aplicadas para suprimir warnings")


# ============================================================================
# CONFIGURAÇÕES GLOBAIS
# ============================================================================

class PipelineConfig:
    """Configurações centralizadas do pipeline"""
    
    # === DADOS ===
    ANO_REFERENCIA = 2024
    SCHEMA = "usr_sat_ecd"
#    TABLE_PLANO_CONTAS = f"{SCHEMA}.ecd_ri050_plano_contas"
    TABLE_BP = f"{SCHEMA}.ecd_rj100_balanco_patrimonial"
    TABLE_DRE = f"{SCHEMA}.ecd_rj150_demonstracao_resultado_exercicio"
    TABLE_IDENTIFICACAO = f"{SCHEMA}.ecd_r0000_identificacao"
    TABLE_REF_BP = "teste.ecd_pc_bp"
    TABLE_REF_DRE = "teste.ecd_pc_dre"
    
    # === PROCESSAMENTO ===
    LIMITE_REGISTROS = None  # Use None para processar todos
    TIPO_CONTA = "A"  # A = Analítica, S = Sintética
    
    # === OUTPUTS ===
    PATH_BASE = "SAT_BIG_DATA/data-pipeline/batch/poc/tsevero"
    PATH_FEATURES = f"{PATH_BASE}/features"
    PATH_ESTRUTURAL = f"{PATH_BASE}/estrutural"
    PATH_DICIONARIOS = f"{PATH_BASE}/dicionarios"
    PATH_GROUND_TRUTH = f"{PATH_BASE}/ground_truth"
    PATH_VALIDACAO = f"{PATH_BASE}/validacao"
    PATH_FINAL = f"{PATH_BASE}/final"
    
    # === CONTROLE DE EXECUÇÃO ===
    EXECUTAR_GROUND_TRUTH = True
    EXECUTAR_FEATURE_ENGINEERING = True
    EXECUTAR_CLASSIFICACAO_ESTRUTURAL = True
    EXECUTAR_CLASSIFICACAO_DICIONARIOS = True
    EXECUTAR_VALIDACAO = True
    SALVAR_INTERMEDIARIOS = True
    
    # === THRESHOLDS GROUND TRUTH ===
    THRESHOLD_MATCH_EXATO = 0.30
    THRESHOLD_MATCH_FUZZY = 0.50
    THRESHOLD_COMPLETUDE = 0.80
    TOP_N_EMPRESAS = 1000
    
    # === TOLERÂNCIAS DE VALIDAÇÃO ===
    TOLERANCIA_EQUACAO = 100.0  # R$ 100 de diferença aceita


# ============================================================================
# MÓDULO 1: DICIONÁRIOS DE PALAVRAS-CHAVE
# ============================================================================

class DicionariosContabeis:
    """
    Dicionários temáticos baseados no padrão da Receita Federal
    Organizados hierarquicamente por categoria contábil
    """
    
    # BALANÇO PATRIMONIAL - ATIVO
    DISPONIBILIDADES = {
        'codigo_rf': '1.01.01',
        'nivel': 3,
        'palavras_chave': [
            'caixa', 'banco', 'bancos', 'conta corrente', 'conta movimento',
            'deposito', 'depositos', 'numerario', 'dinheiro', 'disponivel',
            'disponibilidade', 'aplicacao financeira', 'aplicacoes',
            'fundo liquidez', 'liquidez imediata'
        ],
        'palavras_negativas': ['fornecedor', 'cliente', 'estoque', 'imobilizado']
    }
    
    CLIENTES_DUPLICATAS = {
        'codigo_rf': '1.01.02.02',
        'nivel': 4,
        'palavras_chave': [
            'cliente', 'clientes', 'duplicata', 'duplicatas', 'receber',
            'conta receber', 'contas receber', 'credito', 'creditos',
            'faturamento receber', 'vendas receber', 'titulos receber'
        ],
        'palavras_negativas': ['fornecedor', 'pagar', 'estoque']
    }
    
    ESTOQUES = {
        'codigo_rf': '1.01.03',
        'nivel': 3,
        'palavras_chave': [
            'estoque', 'estoques', 'mercadoria', 'mercadorias',
            'produto', 'produtos', 'materia prima', 'insumo', 'insumos',
            'produto acabado', 'produto processo', 'almoxarifado'
        ],
        'palavras_negativas': ['imobilizado', 'investimento']
    }
    
    TRIBUTOS_RECUPERAR = {
        'codigo_rf': '1.01.02.03',
        'nivel': 4,
        'palavras_chave': [
            'icms recuperar', 'ipi recuperar', 'pis recuperar',
            'cofins recuperar', 'imposto recuperar', 'tributo recuperar',
            'credito tributario', 'credito fiscal'
        ],
        'palavras_negativas': ['pagar', 'recolher', 'provisao']
    }
    
    IMOBILIZADO = {
        'codigo_rf': '1.02.03',
        'nivel': 3,
        'palavras_chave': [
            'imobilizado', 'imovel', 'imoveis', 'veiculo', 'veiculos',
            'maquina', 'maquinas', 'equipamento', 'equipamentos',
            'instalacao', 'instalacoes', 'mobilia', 'moveis', 'utensilio',
            'ferramental', 'edificacao', 'edificacoes', 'terreno', 'terrenos',
            'benfeitoria', 'benfeitorias', 'ativo fixo'
        ],
        'palavras_negativas': []
    }
    
    DEPRECIACAO = {
        'codigo_rf': '1.02.03',
        'nivel': 3,
        'palavras_chave': [
            'depreciacao', 'depreciacao acumulada', 'amortizacao',
            'amortizacao acumulada', 'exaustao', 'exaustao acumulada'
        ],
        'palavras_negativas': []
    }
    
    INVESTIMENTOS = {
        'codigo_rf': '1.02.01',
        'nivel': 3,
        'palavras_chave': [
            'participacao', 'participacoes', 'investimento', 'investimentos',
            'coligada', 'coligadas', 'controlada', 'controladas',
            'acao', 'acoes', 'quota', 'quotas', 'consorcio', 'consorcios'
        ],
        'palavras_negativas': ['capital proprio', 'capital social']
    }
    
    # BALANÇO PATRIMONIAL - PASSIVO
    FORNECEDORES = {
        'codigo_rf': '2.01.03',
        'nivel': 3,
        'palavras_chave': [
            'fornecedor', 'fornecedores', 'duplicata pagar', 'duplicatas pagar',
            'conta pagar', 'contas pagar', 'debito', 'debitos',
            'compra pagar', 'compras pagar', 'obrigacao fornecedor'
        ],
        'palavras_negativas': ['cliente', 'receber']
    }
    
    EMPRESTIMOS_FINANCIAMENTOS = {
        'codigo_rf': '2.01.05',
        'nivel': 3,
        'palavras_chave': [
            'emprestimo', 'emprestimos', 'financiamento', 'financiamentos',
            'banco pagar', 'bancos pagar', 'divida', 'dividas',
            'debito bancario', 'credito bancario', 'linha credito'
        ],
        'palavras_negativas': []
    }
    
    OBRIGACOES_TRABALHISTAS = {
        'codigo_rf': '2.01.02',
        'nivel': 3,
        'palavras_chave': [
            'salario', 'salarios', 'ordenado', 'ordenados', 'folha pagamento',
            'fgts', 'inss', 'ferias', 'decimo terceiro', '13o salario',
            'provisao ferias', 'provisao 13o', 'encargo social',
            'encargos sociais', 'trabalhista', 'trabalhistas'
        ],
        'palavras_negativas': []
    }
    
    OBRIGACOES_FISCAIS = {
        'codigo_rf': '2.01.04',
        'nivel': 3,
        'palavras_chave': [
            'icms pagar', 'ipi pagar', 'pis pagar', 'cofins pagar',
            'iss pagar', 'irpj pagar', 'csll pagar', 'simples pagar',
            'imposto pagar', 'tributo pagar', 'taxa pagar',
            'obrigacao fiscal', 'obrigacao tributaria'
        ],
        'palavras_negativas': ['recuperar', 'credito']
    }
    
    # BALANÇO PATRIMONIAL - PATRIMÔNIO LÍQUIDO
    CAPITAL_SOCIAL = {
        'codigo_rf': '2.03.01',
        'nivel': 3,
        'palavras_chave': [
            'capital social', 'capital subscrito', 'capital integralizado',
            'capital realizado', 'capital'
        ],
        'palavras_negativas': ['capital terceiros', 'capital giro']
    }
    
    RESERVAS = {
        'codigo_rf': '2.03.04',
        'nivel': 3,
        'palavras_chave': [
            'reserva', 'reservas', 'reserva legal', 'reserva lucro',
            'reserva capital', 'reserva estatutaria'
        ],
        'palavras_negativas': []
    }
    
    LUCROS_PREJUIZOS_ACUMULADOS = {
        'codigo_rf': '2.03.09',
        'nivel': 3,
        'palavras_chave': [
            'lucro acumulado', 'lucros acumulados', 'prejuizo acumulado',
            'prejuizos acumulados', 'resultado acumulado', 'lucro exercicio',
            'prejuizo exercicio'
        ],
        'palavras_negativas': []
    }
    
    # DRE - RECEITAS
    RECEITA_BRUTA = {
        'codigo_rf': '3.01.01.01.01',
        'nivel': 5,
        'palavras_chave': [
            'receita bruta', 'receita venda', 'receita vendas',
            'faturamento', 'venda mercadoria', 'venda produto',
            'venda servico', 'receita servico', 'receita operacional'
        ],
        'palavras_negativas': ['deducao', 'devolucao', 'liquida']
    }
    
    DEDUCOES_RECEITA = {
        'codigo_rf': '3.01.01.01.02',
        'nivel': 5,
        'palavras_chave': [
            'deducao', 'deducoes', 'devolucao', 'devolucoes',
            'icms vendas', 'pis vendas', 'cofins vendas',
            'iss vendas', 'abatimento', 'abatimentos',
            'desconto incondicional', 'cancelamento venda'
        ],
        'palavras_negativas': []
    }
    
    # DRE - CUSTOS E DESPESAS
    CUSTO_MERCADORIAS_VENDIDAS = {
        'codigo_rf': '3.01.01.03.01.02',
        'nivel': 6,
        'palavras_chave': [
            'cmv', 'custo mercadoria', 'custo mercadorias',
            'custo produto', 'custo produtos', 'cpv',
            'custo venda', 'custo vendas'
        ],
        'palavras_negativas': ['despesa', 'administrativa', 'comercial']
    }
    
    DESPESAS_ADMINISTRATIVAS = {
        'codigo_rf': '3.01.01.07.01',
        'nivel': 5,
        'palavras_chave': [
            'despesa administrativa', 'despesas administrativas',
            'despesa geral', 'despesas gerais', 'despesa escritorio',
            'material escritorio', 'telefone', 'internet', 'aluguel',
            'condominio', 'agua', 'luz', 'energia eletrica'
        ],
        'palavras_negativas': ['venda', 'comercial']
    }
    
    DESPESAS_COMERCIAIS = {
        'codigo_rf': '3.01.01.07.01',
        'nivel': 5,
        'palavras_chave': [
            'despesa comercial', 'despesas comerciais', 'despesa venda',
            'despesas vendas', 'comissao', 'comissoes', 'propaganda',
            'publicidade', 'marketing', 'frete', 'entrega'
        ],
        'palavras_negativas': []
    }
    
    DESPESAS_PESSOAL = {
        'codigo_rf': '3.01.01.07.01.02',
        'nivel': 6,
        'palavras_chave': [
            'salario', 'salarios', 'ordenado', 'pro labore',
            'honorario', 'honorarios', 'remuneracao', 'gratificacao',
            'despesa pessoal', 'folha pagamento'
        ],
        'palavras_negativas': []
    }
    
    DESPESAS_TRIBUTARIAS = {
        'codigo_rf': '3.01.01.07.01',
        'nivel': 5,
        'palavras_chave': [
            'taxa', 'taxas', 'multa', 'multas', 'tributo', 'tributos',
            'iptu', 'ipva', 'alvara', 'licenca'
        ],
        'palavras_negativas': ['recuperar', 'credito']
    }
    
    DESPESAS_FINANCEIRAS = {
        'codigo_rf': '3.01.01.07.01',
        'nivel': 5,
        'palavras_chave': [
            'juros', 'juro', 'despesa financeira', 'despesas financeiras',
            'iof', 'tarifa bancaria', 'encargo financeiro',
            'despesa bancaria'
        ],
        'palavras_negativas': ['receita']
    }
    
    @classmethod
    def obter_todos_dicionarios(cls):
        """Retorna todos os dicionários como lista"""
        return [
            # Ativo
            cls.DISPONIBILIDADES,
            cls.CLIENTES_DUPLICATAS,
            cls.ESTOQUES,
            cls.TRIBUTOS_RECUPERAR,
            cls.IMOBILIZADO,
            cls.DEPRECIACAO,
            cls.INVESTIMENTOS,
            # Passivo
            cls.FORNECEDORES,
            cls.EMPRESTIMOS_FINANCIAMENTOS,
            cls.OBRIGACOES_TRABALHISTAS,
            cls.OBRIGACOES_FISCAIS,
            # PL
            cls.CAPITAL_SOCIAL,
            cls.RESERVAS,
            cls.LUCROS_PREJUIZOS_ACUMULADOS,
            # DRE
            cls.RECEITA_BRUTA,
            cls.DEDUCOES_RECEITA,
            cls.CUSTO_MERCADORIAS_VENDIDAS,
            cls.DESPESAS_ADMINISTRATIVAS,
            cls.DESPESAS_COMERCIAIS,
            cls.DESPESAS_PESSOAL,
            cls.DESPESAS_TRIBUTARIAS,
            cls.DESPESAS_FINANCEIRAS
        ]


# ============================================================================
# ETAPA 1: CARREGAMENTO DE DADOS
# ============================================================================

def carregar_dados(spark):
    """Carrega dados do plano de contas COM FILTROS DE QUALIDADE"""
    
    print("\n" + "="*80)
    print("📥 ETAPA 1: CARREGAMENTO DE DADOS")
    print("="*80)
    
    # Carregar plano de contas (dados brutos)
    print("\n📊 Carregando dados do Balanço Patrimonial e DRE...")
    
    # 1. Ler Balanço Patrimonial
    df_bp = (spark.table(f"{PipelineConfig.SCHEMA}.ecd_rj100_balanco_patrimonial")
        .filter(col('dt_referencia') >= PipelineConfig.ANO_REFERENCIA)
        .select(
            col('id_ecd'),
            col('dt_referencia'),
            col('cod_agl').alias('cd_conta_anl'),
            col('cod_agl_sup').alias('cd_conta_sint'),
            col('descr_cod_agl'),  # ✅ Nome da conta
            col('ind_dc_cta_fin').alias('cd_natureza'),
            col('nivel_agl').alias('nivel'),
            lit('A').alias('tp_conta'),  # A = Analítica
            lit('BP').alias('origem')  # Marcador de origem
        )
    )
    print(f"  ✅ Balanço Patrimonial: {df_bp.count():,} registros")
    
    # 2. Ler DRE
    df_dre = (spark.table(f"{PipelineConfig.SCHEMA}.ecd_rj150_demonstracao_resultado_exercicio")
        .filter(col('dt_referencia') >= PipelineConfig.ANO_REFERENCIA)
        .select(
            col('id_ecd'),
            col('dt_referencia'),
            col('cod_agl').alias('cd_conta_anl'),
            col('cod_agl_sup').alias('cd_conta_sint'),
            col('descr_cod_agl'),  # ✅ Nome da conta
            col('ind_dc_cta_fin').alias('cd_natureza'),
            col('nivel_agl').alias('nivel'),
            lit('A').alias('tp_conta'),  # A = Analítica
            lit('DRE').alias('origem')  # Marcador de origem
        )
    )
    print(f"  ✅ DRE: {df_dre.count():,} registros")
    
    # 3. UNIÃO das duas tabelas
    df_raw = df_bp.union(df_dre)
    print(f"  ✅ Total após união: {df_raw.count():,} registros")
    
    # 4. Remover duplicatas (mesma conta pode aparecer em BP e DRE)
    print("\n🔄 Removendo duplicatas...")
    df_raw = df_raw.dropDuplicates(['id_ecd', 'dt_referencia', 'cd_conta_anl'])
    print(f"  ✅ Total após deduplicação: {df_raw.count():,} registros")
    
    # ✅ ADICIONAR AQUI: Distribuição por origem
    print("\n📊 Distribuição por origem:")
    df_raw.groupBy('origem').count().orderBy('origem').show()
    
    print("\n🔍 Aplicando filtros de qualidade...")
    
    # Contar registros originais (1x .count())
    qtd_original = df_raw.count()
    print(f"📊 Registros originais: {qtd_original:,}")
    
    # FILTRO 1: Remover códigos/nomes inválidos
    df = df_raw.filter(
        col('cd_conta_anl').isNotNull() & 
        (col('cd_conta_anl') != '') &
        (col('cd_conta_anl') != 'NULL') &
        col('descr_cod_agl').isNotNull() & 
        (col('descr_cod_agl') != '') &
        (col('descr_cod_agl') != 'NULL')
    )
    
    # FILTRO 2: Remover códigos muito curtos
    df = df.filter(length(col('cd_conta_anl')) >= 3)
    
    # FILTRO 3: Remover duplicatas
    if all(c in df.columns for c in ['id_ecd', 'cd_conta_anl', 'dt_referencia']):
        qtd_antes_dedup = df.count()
        df = df.dropDuplicates(['id_ecd', 'cd_conta_anl', 'dt_referencia'])
        qtd_depois_dedup = df.count()
        duplicatas_removidas = qtd_antes_dedup - qtd_depois_dedup
        
        if duplicatas_removidas > 0:
            print(f"   ├─ Duplicatas removidas: {duplicatas_removidas:,}")
    
    # Contar registros finais (1x .count())
    qtd_final = df.count()
    registros_removidos = qtd_original - qtd_final
    
    print(f"📊 Registros filtrados: {qtd_final:,}")
    print(f"🗑️  Registros removidos: {registros_removidos:,}")
    print(f"📈 Taxa de retenção: {(qtd_final / qtd_original * 100):.2f}%")
    
    # Aplicar limite se configurado
    if PipelineConfig.LIMITE_REGISTROS:
        print(f"\n⚠️  Aplicando limite: {PipelineConfig.LIMITE_REGISTROS:,}")
        df = df.limit(PipelineConfig.LIMITE_REGISTROS)
    
    # Cache para performance
    df = df.cache()
    
    # Estatísticas finais
    total = df.count()
    empresas = df.select('id_ecd').distinct().count()
    
    print(f"\n✅ Contas carregadas: {total:,}")
    print(f"✅ Empresas únicas: {empresas:,}")
    print("="*80)
    
    return df

# ============================================================================
# ETAPA 2: FEATURE ENGINEERING
# ============================================================================

def extrair_features(df):
    """Extrai features estruturais do código e nome da conta"""
    
    print("\n" + "="*80)
    print("⚙️  ETAPA 2: FEATURE ENGINEERING")
    print("="*80)
    
    # 1. Primeiro dígito
    df = df.withColumn(
        'primeiro_digito',
        regexp_extract(col('cd_conta_anl'), r'^(\d)', 1)
    )
    
    # 2. Prefixo de 2 caracteres
    df = df.withColumn(
        'prefixo_2_chars',
        upper(substring(col('cd_conta_anl'), 1, 2))
    )
    
    # 3. Tem ponto?
    df = df.withColumn(
        'tem_ponto',
        when(col('cd_conta_anl').contains('.'), 1).otherwise(0)
    )
    
    # 4. Comprimento do código
    df = df.withColumn(
        'comprimento_codigo',
        length(col('cd_conta_anl'))
    )
    
    # 5. Quantidade de pontos
    df = df.withColumn(
        'qtd_pontos',
        length(col('cd_conta_anl')) - length(regexp_replace(col('cd_conta_anl'), r'\.', ''))
    )
    
    # 6. Tem letra?
    df = df.withColumn(
        'tem_letra',
        when(col('cd_conta_anl').rlike('[a-zA-Z]'), 1).otherwise(0)
    )
    
    # 7. Padrão estrutural
    df = df.withColumn(
        'padrao_estrutural',
        when(col('cd_conta_anl').rlike(r'^\d+\.\d+\.\d+'), 'PADRAO_PONTOS')
        .when(col('cd_conta_anl').rlike('^BP'), 'PREFIXO_BP')
        .when(col('cd_conta_anl').rlike('^DRE'), 'PREFIXO_DRE')
        .when(col('cd_conta_anl').rlike(r'^\d+$'), 'SOMENTE_NUMEROS')
        .otherwise('OUTRO')
    )
    
    # 8. Features de natureza
    df = df.withColumn(
        'natureza_devedora',
        when(col('cd_natureza') == '01', 1).otherwise(0)
    ).withColumn(
        'natureza_credora',
        when(col('cd_natureza') == '02', 1).otherwise(0)
    ).withColumn(
        'natureza_resultado',
        when(col('cd_natureza') == '04', 1).otherwise(0)
    )
    
    # 9. Normalizar nome para matching
    df = df.withColumn(
        'nm_conta_normalizado',
        lower(
            regexp_replace(
                regexp_replace(
                    col('descr_cod_agl'),
                    '[áàâãäéèêëíìîïóòôõöúùûüçñ]',
                    ''
                ),
                '[^a-z0-9 ]',
                ''
            )
        )
    )
    
    print("✅ Features extraídas")
    
    # Salvar se configurado
    if PipelineConfig.SALVAR_INTERMEDIARIOS:
        path = PipelineConfig.PATH_FEATURES
        print(f"\n💾 Salvando features em: {path}")
        df.write.mode('overwrite').parquet(path)
    
    return df


# ============================================================================
# ETAPA 3: GROUND TRUTH (EMPRESAS BEM COMPORTADAS)
# ============================================================================

def identificar_ground_truth(spark, df):
    """
    Identifica empresas que seguem rigorosamente o padrão da Receita Federal
    """
    
    print("\n" + "="*80)
    print("🎯 ETAPA 3: IDENTIFICAÇÃO DE GROUND TRUTH")
    print("="*80)
    
    # Carregar tabelas de referência da RF
    try:
        df_ref_bp = spark.table(PipelineConfig.TABLE_REF_BP)
        df_ref_dre = spark.table(PipelineConfig.TABLE_REF_DRE)
        
        print("✅ Tabelas de referência carregadas")
        
        # Verificar colunas disponíveis
        print(f"   Colunas BP:  {df_ref_bp.columns}")
        print(f"   Colunas DRE: {df_ref_dre.columns}")
        
        # Calcular matches exatos (usando nomes corretos das colunas)
        df_matches = df.join(
            df_ref_bp.select(
                col('codigo').alias('cd_conta_anl'),
                col('descricao').alias('desc_rf')
            ),
            on='cd_conta_anl',
            how='left'
        )
        
        # Por empresa, calcular % de matches
        df_stats = df_matches.groupBy('id_ecd', 'dt_referencia').agg(
            count('*').alias('total_contas'),
            spark_sum(when(col('desc_rf').isNotNull(), 1).otherwise(0)).alias('matches_exatos')
        ).withColumn(
            'pct_match',
            col('matches_exatos') / col('total_contas')
        )
        
        # Filtrar empresas bem comportadas
        df_ground_truth = df_stats.filter(
            col('pct_match') >= PipelineConfig.THRESHOLD_MATCH_EXATO
        ).orderBy(desc('pct_match')).limit(PipelineConfig.TOP_N_EMPRESAS)
        
        total_gt = df_ground_truth.count()
        print(f"\n✅ Empresas ground truth identificadas: {total_gt:,}")
        
        # Salvar
        if PipelineConfig.SALVAR_INTERMEDIARIOS:
            path = PipelineConfig.PATH_GROUND_TRUTH
            print(f"💾 Salvando em: {path}")
            df_ground_truth.write.mode('overwrite').parquet(path)
        
        return df_ground_truth
        
    except Exception as e:
        print(f"⚠️  Tabelas de referência não disponíveis: {str(e)}")
        print("   Pulando identificação de ground truth")
        return None


# ============================================================================
# ETAPA 4: CLASSIFICAÇÃO ESTRUTURAL (REGRAS)
# ============================================================================

def classificar_estrutural(df):
    """Aplica regras para classificar contas baseado em padrões estruturais"""
    
    print("\n" + "="*80)
    print("📋 ETAPA 4: CLASSIFICAÇÃO ESTRUTURAL (REGRAS)")
    print("="*80)
    
    # --- NÍVEL 1: CLASSES (ATIVO, PASSIVO, RESULTADO) ---
    print("   → Nível 1: Classes (ATIVO, PASSIVO, RESULTADO)...")
    
    # Classificação Nível 1
    df = df.withColumn(
        'classificacao_nivel_1',
        # Regra 1: Natureza + Nível exato
        when((col('cd_natureza') == '01') & (col('nivel').cast('int') == 1), lit(1))  # ATIVO
        .when((col('cd_natureza') == '02') & (col('nivel').cast('int') == 1), lit(2))  # PASSIVO
        .when(col('cd_natureza') == '04', lit(3))  # RESULTADO
        # Regra 2: Primeiro dígito
        .when(col('primeiro_digito') == '1', lit(1))  # ATIVO
        .when(col('prefixo_2_chars') == '01', lit(1))  # ATIVO
        .when(col('primeiro_digito') == '2', lit(2))  # PASSIVO
        .when(col('prefixo_2_chars') == '02', lit(2))  # PASSIVO
        .when(col('primeiro_digito') == '3', lit(3))  # RESULTADO
        .when(col('prefixo_2_chars') == '03', lit(3))  # RESULTADO
        .otherwise(None)
    )
    
    # Descrição da classificação
    df = df.withColumn(
        'classificacao_nivel_1_desc',
        when(col('classificacao_nivel_1') == 1, 'ATIVO')
        .when(col('classificacao_nivel_1') == 2, 'PASSIVO')
        .when(col('classificacao_nivel_1') == 3, 'RESULTADO')
        .otherwise(None)
    )
    
    # Confiança da classificação
    df = df.withColumn(
        'confianca_nivel_1',
        when((col('cd_natureza') == '01') & (col('nivel').cast('int') == 1), lit(0.95))
        .when((col('cd_natureza') == '02') & (col('nivel').cast('int') == 1), lit(0.95))
        .when(col('cd_natureza') == '04', lit(0.90))
        .when(col('primeiro_digito').isin(['1', '2', '3']), lit(0.80))
        .otherwise(0.0)
    )
    
    # --- NÍVEL 2: SUBGRUPOS ---
    print("   → Nível 2: Subgrupos (Circulante, Não Circulante, etc)...")
    
    df = df.withColumn(
        'classificacao_nivel_2',
        # ATIVO
        when((col('classificacao_nivel_1') == 1) & 
             (col('cd_conta_anl').rlike(r'^1\.?01') | col('cd_conta_anl').rlike(r'^01\.?1\.?01')), 
             lit(1.01))  # Ativo Circulante
        .when((col('classificacao_nivel_1') == 1) & 
              (col('cd_conta_anl').rlike(r'^1\.?02') | col('cd_conta_anl').rlike(r'^01\.?1\.?02')), 
              lit(1.02))  # Ativo Não Circulante
        # PASSIVO
        .when((col('classificacao_nivel_1') == 2) & 
              (col('cd_conta_anl').rlike(r'^2\.?01') | col('cd_conta_anl').rlike(r'^02\.?1\.?01')), 
              lit(2.01))  # Passivo Circulante
        .when((col('classificacao_nivel_1') == 2) & 
              (col('cd_conta_anl').rlike(r'^2\.?02') | col('cd_conta_anl').rlike(r'^02\.?1\.?02')), 
              lit(2.02))  # Passivo Não Circulante
        .when((col('classificacao_nivel_1') == 2) & 
              (col('cd_conta_anl').rlike(r'^2\.?03') | col('cd_conta_anl').rlike(r'^02\.?3')), 
              lit(2.03))  # Patrimônio Líquido
        .otherwise(col('classificacao_nivel_1'))  # Manter nível 1 se não houver nível 2
    )
    
    # Adicionar método de classificação
    df = df.withColumn(
        'metodo_classificacao',
        when(col('classificacao_nivel_1').isNotNull(), 'ESTRUTURAL')
        .otherwise(None)
    )
    
    # Estatísticas
    total = df.count()
    n1 = df.filter(col('classificacao_nivel_1').isNotNull()).count()
    n2 = df.filter(col('classificacao_nivel_2').isNotNull()).count()
    
    print(f"\n✅ Nível 1: {n1:,} / {total:,} ({n1/total*100:.1f}%)")
    print(f"✅ Nível 2: {n2:,} / {total:,} ({n2/total*100:.1f}%)")
    
    # Salvar se configurado
    if PipelineConfig.SALVAR_INTERMEDIARIOS:
        path = PipelineConfig.PATH_ESTRUTURAL
        print(f"\n💾 Salvando classificação estrutural em: {path}")
        df.write.mode('overwrite').parquet(path)
    
    return df


# ============================================================================
# ETAPA 5: CLASSIFICAÇÃO POR DICIONÁRIOS
# ============================================================================

def normalizar_texto_classificacao(texto):
    """
    Normaliza texto para matching de palavras-chave
    """
    if texto is None:
        return ""
    
    # Remove acentos e caracteres especiais
    texto = texto.lower()
    texto = texto.replace('á', 'a').replace('à', 'a').replace('â', 'a').replace('ã', 'a')
    texto = texto.replace('é', 'e').replace('è', 'e').replace('ê', 'e')
    texto = texto.replace('í', 'i').replace('ì', 'i').replace('î', 'i')
    texto = texto.replace('ó', 'o').replace('ò', 'o').replace('ô', 'o').replace('õ', 'o')
    texto = texto.replace('ú', 'u').replace('ù', 'u').replace('û', 'u')
    texto = texto.replace('ç', 'c').replace('ñ', 'n')
    
    # Remove caracteres especiais, mantendo apenas letras e espaços
    import re
    texto = re.sub(r'[^a-z0-9 ]', '', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()
    
    return texto


def classificar_por_dicionarios(df):
    """
    Classifica contas usando dicionários de palavras-chave
    Aplicado apenas às contas que não foram classificadas por regras estruturais
    
    VERSÃO OTIMIZADA: Processa dicionários em lotes para evitar erros de compilação
    """
    
    print("\n" + "="*80)
    print("📚 ETAPA 5: CLASSIFICAÇÃO POR DICIONÁRIOS")
    print("="*80)
    
    # Identificar contas não classificadas
    df_nao_classificadas = df.filter(col('classificacao_nivel_1').isNull())
    df_classificadas = df.filter(col('classificacao_nivel_1').isNotNull())
    
    total_nao_class = df_nao_classificadas.count()
    print(f"\n📊 Contas não classificadas: {total_nao_class:,}")
    
    if total_nao_class == 0:
        print("✅ Todas as contas já foram classificadas!")
        return df
    
    # Obter todos os dicionários
    dicionarios = DicionariosContabeis.obter_todos_dicionarios()
    print(f"✅ Dicionários carregados: {len(dicionarios)}")
    
    # ESTRATÉGIA OTIMIZADA: Processar em lotes menores
    TAMANHO_LOTE = 5  # Processar 5 dicionários por vez
    df_dict_class = df_nao_classificadas
    
    for lote_idx in range(0, len(dicionarios), TAMANHO_LOTE):
        lote = dicionarios[lote_idx:lote_idx + TAMANHO_LOTE]
        print(f"\n   Processando lote {lote_idx//TAMANHO_LOTE + 1}/{(len(dicionarios)-1)//TAMANHO_LOTE + 1}...")
        
        for i, dicionario in enumerate(lote):
            idx_global = lote_idx + i
            
            # Criar lista de palavras-chave para usar com SQL LIKE
            palavras_positivas = [normalizar_texto_classificacao(p) for p in dicionario['palavras_chave']]
            palavras_negativas = [normalizar_texto_classificacao(p) for p in dicionario['palavras_negativas']]
            
            # Construir condição usando array_contains (mais eficiente)
            codigo_rf = dicionario['codigo_rf']
            
            # Condição positiva simplificada
            condicao = lit(False)
            for palavra in palavras_positivas[:10]:  # Limitar a 10 palavras por dicionário
                condicao = condicao | col('nm_conta_normalizado').contains(palavra)
            
            # Condição negativa
            if palavras_negativas:
                for palavra in palavras_negativas[:5]:  # Limitar a 5 palavras negativas
                    condicao = condicao & ~col('nm_conta_normalizado').contains(palavra)
            
            df_dict_class = df_dict_class.withColumn(
                f'match_dict_{idx_global}',
                when(condicao, lit(codigo_rf)).otherwise(None)
            )
        
        # A cada lote, consolidar para evitar acúmulo excessivo de colunas
        if (lote_idx + TAMANHO_LOTE) % 10 == 0:
            match_cols_parcial = [f'match_dict_{j}' for j in range(lote_idx + TAMANHO_LOTE)]
            df_dict_class = df_dict_class.withColumn(
                f'consolidado_{lote_idx}',
                coalesce(*[col(c) for c in match_cols_parcial if c in df_dict_class.columns])
            )
            # Remover colunas intermediárias
            for c in match_cols_parcial:
                if c in df_dict_class.columns:
                    df_dict_class = df_dict_class.drop(c)
    
    # Consolidar todas as classificações
    cols_consolidadas = [c for c in df_dict_class.columns if c.startswith('consolidado_')]
    match_cols_finais = [c for c in df_dict_class.columns if c.startswith('match_dict_')]
    
    todas_cols = cols_consolidadas + match_cols_finais
    if todas_cols:
        df_dict_class = df_dict_class.withColumn(
            'classificacao_dicionario',
            coalesce(*[col(c) for c in todas_cols])
        )
    else:
        df_dict_class = df_dict_class.withColumn('classificacao_dicionario', lit(None))
    
    # Atualizar classificação final
    df_dict_class = df_dict_class.withColumn(
        'classificacao_nivel_1',
        when(col('classificacao_dicionario').isNotNull(),
             substring(col('classificacao_dicionario'), 1, 1).cast(IntegerType()))
        .otherwise(col('classificacao_nivel_1'))
    ).withColumn(
        'classificacao_nivel_1_desc',
        when(col('classificacao_dicionario').isNotNull(),
             when(col('classificacao_dicionario').startswith('1'), 'ATIVO')
             .when(col('classificacao_dicionario').startswith('2'), 'PASSIVO')
             .when(col('classificacao_dicionario').startswith('3'), 'RESULTADO')
             .otherwise(None))
        .otherwise(col('classificacao_nivel_1_desc'))
    ).withColumn(
        'classificacao_nivel_2',
        when(col('classificacao_dicionario').isNotNull(), col('classificacao_dicionario'))
        .otherwise(col('classificacao_nivel_2'))
    ).withColumn(
        'metodo_classificacao',
        when(col('classificacao_dicionario').isNotNull(), 'DICIONARIO')
        .otherwise(col('metodo_classificacao'))
    ).withColumn(
        'confianca_nivel_1',
        when(col('classificacao_dicionario').isNotNull(), lit(0.75))
        .otherwise(col('confianca_nivel_1'))
    )
    
    # Limpar todas as colunas temporárias
    colunas_para_remover = ([c for c in df_dict_class.columns if c.startswith('match_dict_')] +
                           [c for c in df_dict_class.columns if c.startswith('consolidado_')] +
                           ['classificacao_dicionario'])
    
    for col_name in colunas_para_remover:
        if col_name in df_dict_class.columns:
            df_dict_class = df_dict_class.drop(col_name)
    
    # Juntar de volta com as já classificadas
    df_final = df_classificadas.unionByName(df_dict_class, allowMissingColumns=True)
    
    # Estatísticas
    classificadas_dict = df_dict_class.filter(col('metodo_classificacao') == 'DICIONARIO').count()
    print(f"\n✅ Classificadas por dicionário: {classificadas_dict:,}")
    
    # Salvar
    if PipelineConfig.SALVAR_INTERMEDIARIOS:
        path = PipelineConfig.PATH_DICIONARIOS
        print(f"💾 Salvando em: {path}")
        df_final.write.mode('overwrite').parquet(path)
    
    return df_final


# ============================================================================
# ETAPA 6: CONSOLIDAÇÃO E CLASSIFICAÇÃO FINAL
# ============================================================================

def consolidar_classificacao(df):
    """
    Consolida todas as classificações em uma classificação final
    """
    
    print("\n" + "="*80)
    print("🔄 ETAPA 6: CONSOLIDAÇÃO DA CLASSIFICAÇÃO")
    print("="*80)
    
    # Criar classificação final (prioridade: estrutural > dicionário)
    df = df.withColumn(
        'classificacao_final',
        coalesce(
            col('classificacao_nivel_2'),
            col('classificacao_nivel_1').cast(StringType())
        )
    )
    
    # Confiança final
    df = df.withColumn(
        'confianca_final',
        coalesce(col('confianca_nivel_1'), lit(0.0))
    )
    
    # Estatísticas finais
    total = df.count()
    classificadas = df.filter(col('classificacao_final').isNotNull()).count()
    
    print(f"\n📊 RESULTADO FINAL:")
    print(f"   Total de contas:       {total:,}")
    print(f"   Contas classificadas:  {classificadas:,} ({classificadas/total*100:.1f}%)")
    print(f"   Não classificadas:     {total - classificadas:,}")
    
    # Distribuição por método
    print("\n📊 Distribuição por método de classificação:")
    df.filter(col('classificacao_final').isNotNull()) \
        .groupBy('metodo_classificacao').count() \
        .orderBy(desc('count')).show(truncate=False)
    
    return df


# ============================================================================
# ETAPA 7: VALIDAÇÃO CONTÁBIL
# ============================================================================

def validar_equacao_contabil(spark, df_classificado):
    """
    Valida a equação contábil fundamental: ATIVO = PASSIVO + PL
    VERSÃO CORRIGIDA com tratamento de NULLs
    """
    
    print("\n" + "="*80)
    print("✅ ETAPA 7: VALIDAÇÃO CONTÁBIL")
    print("="*80)
    print("\n🔍 VALIDAÇÃO 1: EQUAÇÃO CONTÁBIL (ATIVO = PASSIVO + PL)")
    
    # ========================================================================
    # VERIFICAÇÃO PRÉVIA: Checar se as colunas necessárias existem
    # ========================================================================
    
    colunas_necessarias = ['id_ecd', 'dt_referencia', 'cd_conta_anl', 'classificacao_final']
    colunas_faltando = [c for c in colunas_necessarias if c not in df_classificado.columns]
    
    if colunas_faltando:
        print(f"❌ ERRO: Colunas faltando no DataFrame classificado: {colunas_faltando}")
        print(f"   Colunas disponíveis: {df_classificado.columns}")
        
        # Tentar usar classificacao_nivel_2 como fallback
        if 'classificacao_nivel_2' in df_classificado.columns:
            print("   ⚠️ Usando 'classificacao_nivel_2' como fallback")
            df_classificado = df_classificado.withColumn(
                'classificacao_final',
                coalesce(
                    col('classificacao_nivel_2'),
                    col('classificacao_nivel_1').cast(StringType())
                )
            )
        else:
            print("   ❌ Impossível continuar validação sem classificação")
            return None
    
    # ========================================================================
    # CARREGAR BALANÇO PATRIMONIAL
    # ========================================================================
    
    print(f"\n📊 Carregando dados de {PipelineConfig.TABLE_BP}...")
    
    df_bp = spark.sql(f"""
        SELECT 
            id_ecd, 
            dt_referencia, 
            cod_agl, 
            ind_grp_bal,
            vl_cta_fin, 
            ind_dc_cta_fin
        FROM {PipelineConfig.TABLE_BP}
        WHERE dt_referencia >= {PipelineConfig.ANO_REFERENCIA}
          AND ind_cod_agl = 'D'
          AND vl_cta_fin IS NOT NULL
    """)
    
    total_bp = df_bp.count()
    print(f"   ✅ Registros do BP: {total_bp:,}")
    
    if total_bp == 0:
        print("   ❌ Nenhum registro encontrado no Balanço Patrimonial!")
        return None
    
    # ========================================================================
    # JUNTAR COM CLASSIFICAÇÃO
    # ========================================================================
    
    print(f"\n🔄 Juntando BP com classificações...")
    
    df_bp_class = df_bp.join(
        df_classificado.select(
            'id_ecd', 
            'dt_referencia',
            col('cd_conta_anl').alias('cod_agl'),
            'classificacao_final'
        ).filter(col('classificacao_final').isNotNull()),  # ✅ Filtrar NULLs
        on=['id_ecd', 'dt_referencia', 'cod_agl'],
        how='left'
    )
    
    # Estatísticas de JOIN
    total_com_class = df_bp_class.filter(col('classificacao_final').isNotNull()).count()
    pct_class = (total_com_class / total_bp * 100) if total_bp > 0 else 0
    
    print(f"   ✅ Contas com classificação: {total_com_class:,} ({pct_class:.1f}%)")
    print(f"   ⚠️ Contas sem classificação: {total_bp - total_com_class:,}")
    
    # ========================================================================
    # DETERMINAR GRUPO CONTÁBIL
    # ========================================================================
    
    df_bp_class = df_bp_class.withColumn(
        'grupo_contabil',
        when(col('classificacao_final').isNull(), lit('SEM_CLASSIFICACAO'))
        .when(col('classificacao_final').startswith('1'), lit('ATIVO'))
        .when(col('classificacao_final').startswith('2.01'), lit('PASSIVO'))
        .when(col('classificacao_final').startswith('2.02'), lit('PASSIVO'))
        .when(col('classificacao_final').startswith('2.03'), lit('PL'))
        .otherwise(lit('OUTROS'))
    )
    
    # Verificar distribuição
    print("\n📊 Distribuição por Grupo Contábil:")
    df_bp_class.groupBy('grupo_contabil').count().orderBy(desc('count')).show()
    
    # ========================================================================
    # AJUSTAR VALOR PELO INDICADOR D/C
    # ========================================================================
    
    df_bp_class = df_bp_class.withColumn(
        'valor_ajustado',
        when(col('ind_dc_cta_fin') == 'D', col('vl_cta_fin'))
        .when(col('ind_dc_cta_fin') == 'C', -col('vl_cta_fin'))
        .otherwise(lit(0))
    )
    
    # ========================================================================
    # TOTALIZAR POR EMPRESA
    # ========================================================================
    
    print("\n📊 Calculando totais por empresa...")
    
    df_totais = df_bp_class.groupBy('id_ecd', 'dt_referencia').agg(
        spark_sum(
            when(col('grupo_contabil') == 'ATIVO', col('valor_ajustado'))
            .otherwise(0)
        ).alias('total_ativo'),
        spark_sum(
            when(col('grupo_contabil') == 'PASSIVO', col('valor_ajustado'))
            .otherwise(0)
        ).alias('total_passivo'),
        spark_sum(
            when(col('grupo_contabil') == 'PL', col('valor_ajustado'))
            .otherwise(0)
        ).alias('total_pl'),
        spark_sum(
            when(col('grupo_contabil') == 'OUTROS', col('valor_ajustado'))
            .otherwise(0)
        ).alias('total_outros'),
        spark_sum(
            when(col('grupo_contabil') == 'SEM_CLASSIFICACAO', col('valor_ajustado'))
            .otherwise(0)
        ).alias('total_sem_class')
    )
    
    # ========================================================================
    # CALCULAR DIFERENÇA E VALIDAR
    # ========================================================================
    
    df_totais = df_totais.withColumn(
        'passivo_pl_calculado',
        coalesce(col('total_passivo'), lit(0)) + coalesce(col('total_pl'), lit(0))
    ).withColumn(
        'diferenca',
        spark_abs(col('total_ativo') - col('passivo_pl_calculado'))
    ).withColumn(
        'diferenca_percentual',
        when(col('total_ativo') != 0,
             (col('diferenca') / spark_abs(col('total_ativo'))) * 100)
        .otherwise(lit(0))
    ).withColumn(
        'equacao_valida',
        (col('diferenca') <= PipelineConfig.TOLERANCIA_EQUACAO) |
        (col('diferenca_percentual') <= 0.01)  # Tolerância de 0.01%
    )
    
    # ========================================================================
    # ESTATÍSTICAS FINAIS
    # ========================================================================
    
    total_empresas = df_totais.count()
    empresas_validas = df_totais.filter(col('equacao_valida')).count()
    pct_validas = (empresas_validas / total_empresas * 100) if total_empresas > 0 else 0
    
    print(f"\n📊 RESULTADOS DA VALIDAÇÃO:")
    print(f"   Total de empresas:     {total_empresas:,}")
    print(f"   Equação válida:        {empresas_validas:,} ({pct_validas:.1f}%)")
    print(f"   Equação inválida:      {total_empresas - empresas_validas:,}")
    
    # Top 10 maiores diferenças
    print("\n⚠️ Top 10 maiores diferenças:")
    df_totais.orderBy(desc('diferenca')).select(
        'id_ecd',
        'dt_referencia',
        spark_round('total_ativo', 2).alias('ativo'),
        spark_round('passivo_pl_calculado', 2).alias('passivo_pl'),
        spark_round('diferenca', 2).alias('diferenca'),
        spark_round('diferenca_percentual', 2).alias('diferenca_%'),
        'equacao_valida'
    ).show(10, truncate=False)
    
    # Estatísticas de valores não classificados
    df_stats_outros = df_totais.select(
        spark_sum('total_outros').alias('soma_outros'),
        spark_sum('total_sem_class').alias('soma_sem_class'),
        spark_sum(spark_abs('total_ativo')).alias('soma_ativos')
    ).collect()[0]
    
    if df_stats_outros['soma_ativos'] > 0:
        pct_outros = (df_stats_outros['soma_outros'] / df_stats_outros['soma_ativos']) * 100
        pct_sem_class = (df_stats_outros['soma_sem_class'] / df_stats_outros['soma_ativos']) * 100
        
        print(f"\n⚠️ Impacto de contas não classificadas:")
        print(f"   Classificadas como 'OUTROS': {pct_outros:.2f}% do ativo total")
        print(f"   Sem classificação: {pct_sem_class:.2f}% do ativo total")
    
    # ========================================================================
    # SALVAR RESULTADOS
    # ========================================================================
    
    if PipelineConfig.SALVAR_INTERMEDIARIOS:
        path = f"{PipelineConfig.PATH_VALIDACAO}/equacao_contabil"
        print(f"\n💾 Salvando validação em: {path}")
        df_totais.write.mode('overwrite').parquet(path)
    
    print("\n" + "="*80)
    
    return df_totais

def validacao_simples(df_final):
    """
    Validação básica dos resultados de classificação
    """
    
    print("\n" + "="*80)
    print("📋 VALIDAÇÃO SIMPLES DA CLASSIFICAÇÃO")
    print("="*80)
    
    # 1. Distribuição por classificação nível 1
    print("\n📊 Distribuição Nível 1:")
    df_final.groupBy('classificacao_nivel_1_desc').count() \
        .orderBy('classificacao_nivel_1_desc').show()
    
    # 2. Distribuição por classificação nível 2
    print("\n📊 Distribuição Nível 2 (Top 10):")
    df_final.filter(col('classificacao_nivel_2').isNotNull()) \
        .groupBy('classificacao_nivel_2').count() \
        .orderBy(desc('count')).show(10)
    
    # 3. Contas não classificadas
    print("\n⚠️ Contas NÃO classificadas (Nível 1):")
    nao_classificadas = df_final.filter(col('classificacao_nivel_1').isNull())
    count_nao_class = nao_classificadas.count()
    
    if count_nao_class > 0:
        print(f"   Total: {count_nao_class:,}")
        print("\n   Exemplos:")
        nao_classificadas.select(
            'cd_conta_anl',
            'descr_cod_agl',
            'cd_natureza',
            'primeiro_digito'
        ).show(10, truncate=False)
    else:
        print("   ✅ Todas as contas foram classificadas!")
    
    # 4. Distribuição de confiança
    print("\n📊 Distribuição de Confiança:")
    df_final.select('confianca_final').summary().show()
    
    # 5. Faixas de confiança
    print("\n📊 Classificações por faixa de confiança:")
    df_final.filter(col('classificacao_nivel_1').isNotNull()) \
        .select(
            when(col('confianca_final') >= 0.90, 'ALTA (≥0.90)')
            .when(col('confianca_final') >= 0.80, 'BOA (0.80-0.89)')
            .when(col('confianca_final') >= 0.70, 'MODERADA (0.70-0.79)')
            .otherwise('BAIXA (<0.70)')
            .alias('faixa')
        ).groupBy('faixa').count().orderBy('faixa').show(truncate=False)
    
    return {
        'total': df_final.count(),
        'classificadas_n1': df_final.filter(col('classificacao_nivel_1').isNotNull()).count(),
        'classificadas_n2': df_final.filter(col('classificacao_nivel_2').isNotNull()).count(),
        'nao_classificadas': count_nao_class
    }


# ============================================================================
# ORQUESTRADOR DO PIPELINE
# ============================================================================

def executar_pipeline_completo(spark):
    """Executa o pipeline completo de classificação"""
    
    print("\n" + "="*80)
    print("🚀 INICIANDO PIPELINE COMPLETO DE CLASSIFICAÇÃO ECD")
    print("="*80)
    print(f"Data/Hora: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Ano de referência: {PipelineConfig.ANO_REFERENCIA}")
    print("="*80)
    
    inicio = datetime.now()
    resultados = {}
    
    try:
        # 1. Carregamento
        df = carregar_dados(spark)
        
        # 2. Feature Engineering
        if PipelineConfig.EXECUTAR_FEATURE_ENGINEERING:
            df = extrair_features(df)
        
        # 3. Ground Truth
        if PipelineConfig.EXECUTAR_GROUND_TRUTH:
            try:
                df_ground_truth = identificar_ground_truth(spark, df)
                resultados['ground_truth'] = df_ground_truth
            except Exception as e:
                print(f"⚠️  Erro em Ground Truth: {str(e)}")
        
        # 4. Classificação Estrutural
        if PipelineConfig.EXECUTAR_CLASSIFICACAO_ESTRUTURAL:
            df = classificar_estrutural(df)
        
        # 5. Classificação por Dicionários
        if PipelineConfig.EXECUTAR_CLASSIFICACAO_DICIONARIOS:
            df = classificar_por_dicionarios(df)
        
        # 6. Consolidação
        df_final = consolidar_classificacao(df)
        
        # 7. Validação
        if PipelineConfig.EXECUTAR_VALIDACAO:
            print("\n✅ Executando validação...")
            try:
                # Validação simples
                resultado_validacao = validacao_simples(df_final)
                resultados['validacao_simples'] = resultado_validacao
                
                # Validação da equação contábil
                df_equacao = validar_equacao_contabil(spark, df_final)
                resultados['validacao_equacao'] = df_equacao
                
            except Exception as e:
                print(f"⚠️  Erro na Validação: {str(e)}")
        
        # Relatório Final
        fim = datetime.now()
        duracao = fim - inicio
        
        print("\n" + "="*80)
        print("✅ PIPELINE CONCLUÍDO COM SUCESSO")
        print("="*80)
        print(f"Início:   {inicio.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Fim:      {fim.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Duração:  {duracao}")
        print("="*80)
        
        print("\n📊 ESTATÍSTICAS FINAIS:")
        total = df_final.count()
        classificadas_n1 = df_final.filter(col('classificacao_nivel_1').isNotNull()).count()
        classificadas_n2 = df_final.filter(col('classificacao_nivel_2').isNotNull()).count()
        
        print(f"   Total de contas:       {total:,}")
        print(f"   Classificadas Nível 1: {classificadas_n1:,} ({classificadas_n1/total*100:.1f}%)")
        print(f"   Classificadas Nível 2: {classificadas_n2:,} ({classificadas_n2/total*100:.1f}%)")
        
        # Salvar resultado final
        if PipelineConfig.SALVAR_INTERMEDIARIOS:
            print(f"\n💾 Salvando resultado final em: {PipelineConfig.PATH_FINAL}")
            df_final.write.mode('overwrite').parquet(PipelineConfig.PATH_FINAL)
        
        return {
            'df_final': df_final,
            'resultados': resultados,
            'estatisticas': {
                'total': total,
                'classificadas_n1': classificadas_n1,
                'classificadas_n2': classificadas_n2,
                'duracao': duracao
            }
        }
        
    except Exception as e:
        print(f"\n❌ ERRO NO PIPELINE: {str(e)}")
        import traceback
        traceback.print_exc()
        return None


# ============================================================================
# FUNÇÕES AUXILIARES
# ============================================================================

def carregar_resultado_intermediario(spark, etapa):
    """Carrega resultado de uma etapa já processada"""
    
    path_map = {
        'features': PipelineConfig.PATH_FEATURES,
        'estrutural': PipelineConfig.PATH_ESTRUTURAL,
        'dicionarios': PipelineConfig.PATH_DICIONARIOS,
        'ground_truth': PipelineConfig.PATH_GROUND_TRUTH,
        'final': PipelineConfig.PATH_FINAL
    }
    
    path = path_map.get(etapa, f"{PipelineConfig.PATH_BASE}/{etapa}")
    print(f"📥 Carregando resultado intermediário de: {path}")
    
    try:
        df = spark.read.parquet(path)
        print(f"✅ Carregado: {df.count():,} registros")
        return df
    except Exception as e:
        print(f"❌ Erro ao carregar: {str(e)}")
        return None


def exportar_para_tabela_hive(df, nome_tabela, particionar_por=None):
    """Exporta resultado para tabela Hive"""
    
    print(f"\n💾 Exportando para tabela Hive: {nome_tabela}")
    
    writer = df.write.mode('overwrite')
    
    if particionar_por:
        writer = writer.partitionBy(particionar_por)
    
    writer.saveAsTable(nome_tabela)
    
    print(f"✅ Tabela criada: {nome_tabela}")


# ============================================================================
# MAIN
# ============================================================================

def main(spark):
    """Função principal"""
    
    resultado = executar_pipeline_completo(spark)
    
    return resultado


# ============================================================================
# EXECUÇÃO
# ============================================================================

if __name__ == "__main__":
    print("""
    ================================================================================
    PIPELINE COMPLETO DE CLASSIFICAÇÃO CONTÁBIL ECD
    ================================================================================
    
    Para executar:
    
    1. No ambiente PySpark:
       >>> resultado = main(spark)
    
    2. Acessar resultados:
       >>> df_final = resultado['df_final']
       >>> estatisticas = resultado['estatisticas']
    
    3. Carregar resultados intermediários:
       >>> df = carregar_resultado_intermediario(spark, 'final')
    
    4. Exportar para Hive:
       >>> exportar_para_tabela_hive(
       ...     resultado['df_final'], 
       ...     'usr_sat_ecd.contas_classificadas',
       ...     particionar_por='dt_referencia'
       ... )
    
    ================================================================================
    """)

✅ Configurações aplicadas para suprimir warnings

    PIPELINE COMPLETO DE CLASSIFICAÇÃO CONTÁBIL ECD
    
    Para executar:
    
    1. No ambiente PySpark:
       >>> resultado = main(spark)
    
    2. Acessar resultados:
       >>> df_final = resultado['df_final']
       >>> estatisticas = resultado['estatisticas']
    
    3. Carregar resultados intermediários:
       >>> df = carregar_resultado_intermediario(spark, 'final')
    
    4. Exportar para Hive:
       >>> exportar_para_tabela_hive(
       ...     resultado['df_final'], 
       ...     'usr_sat_ecd.contas_classificadas',
       ...     particionar_por='dt_referencia'
       ... )
    
    


In [7]:
"""
================================================================================
CÉLULA DE FILTRO DE DADOS - ADICIONAR LOGO APÓS CONFIGURAÇÃO INICIAL
================================================================================

INSTRUÇÕES:
1. Adicione esta célula DEPOIS da célula de configuração inicial
2. ANTES de qualquer processamento de dados
3. Esta célula deve ser a primeira a carregar e filtrar os dados

POSIÇÃO RECOMENDADA:
- Depois da célula com "class PipelineConfig"
- Antes de qualquer célula que carregue dados das tabelas

================================================================================
"""

from pyspark.sql.functions import col, abs as spark_abs, coalesce, lit

# ============================================================================
# FUNÇÃO DE FILTRO GLOBAL - REMOVER ZEROS E NULOS
# ============================================================================

def filtrar_dados_validos(df, campos_saldo=['saldo_final_contabil', 'saldo_inicial_contabil', 
                                              'valor_debito_periodo', 'valor_credito_periodo']):
    """
    Filtra registros com valores zerados ou nulos nos campos de saldo.
    
    Args:
        df: DataFrame a ser filtrado
        campos_saldo: Lista de campos de saldo para verificar
        
    Returns:
        DataFrame filtrado (sem zeros e nulos)
    """
    
    print(f"📊 Registros antes do filtro: {df.count():,}")
    
    # Construir condição de filtro
    # Remove registros onde TODOS os campos de saldo são zero ou nulo
    condicao_valida = None
    
    for campo in campos_saldo:
        if campo in df.columns:
            # Campo não é nulo E não é zero
            condicao_campo = (
                col(campo).isNotNull() & 
                (spark_abs(col(campo)) > 0.01)  # Tolerância para valores muito pequenos
            )
            
            if condicao_valida is None:
                condicao_valida = condicao_campo
            else:
                # Pelo menos UM campo deve ter valor válido
                condicao_valida = condicao_valida | condicao_campo
    
    # Aplicar filtro
    if condicao_valida is not None:
        df_filtrado = df.filter(condicao_valida)
    else:
        df_filtrado = df
    
    registros_removidos = df.count() - df_filtrado.count()
    print(f"📊 Registros depois do filtro: {df_filtrado.count():,}")
    print(f"🗑️  Registros removidos (zeros/nulos): {registros_removidos:,}")
    print(f"✅ Taxa de retenção: {(df_filtrado.count() / df.count() * 100):.2f}%")
    
    return df_filtrado


def filtrar_contas_validas(df, campo_conta='cd_conta', campo_nome='nm_conta'):
    """
    Remove registros com códigos ou nomes de conta inválidos.
    
    Args:
        df: DataFrame a ser filtrado
        campo_conta: Nome do campo com código da conta
        campo_nome: Nome do campo com nome da conta
        
    Returns:
        DataFrame filtrado
    """
    
    print(f"\n📋 Filtrando contas inválidas...")
    print(f"📊 Registros antes: {df.count():,}")
    
    condicoes = []
    
    # 1. Código da conta não pode ser nulo ou vazio
    if campo_conta in df.columns:
        condicoes.append(
            col(campo_conta).isNotNull() & 
            (col(campo_conta) != '') &
            (col(campo_conta) != 'NULL')
        )
    
    # 2. Nome da conta não pode ser nulo ou vazio
    if campo_nome in df.columns:
        condicoes.append(
            col(campo_nome).isNotNull() & 
            (col(campo_nome) != '') &
            (col(campo_nome) != 'NULL')
        )
    
    # Aplicar todas as condições
    if condicoes:
        condicao_final = condicoes[0]
        for cond in condicoes[1:]:
            condicao_final = condicao_final & cond
        
        df_filtrado = df.filter(condicao_final)
    else:
        df_filtrado = df
    
    registros_removidos = df.count() - df_filtrado.count()
    print(f"📊 Registros depois: {df_filtrado.count():,}")
    print(f"🗑️  Registros removidos (inválidos): {registros_removidos:,}")
    
    return df_filtrado


def aplicar_filtros_globais(df, tipo_tabela='saldos'):
    """
    Aplica todos os filtros de qualidade nos dados.
    
    Args:
        df: DataFrame a ser filtrado
        tipo_tabela: Tipo de tabela ('saldos', 'plano_contas', 'balanco', 'dre')
        
    Returns:
        DataFrame filtrado e limpo
    """
    
    print("="*80)
    print(f"🔍 APLICANDO FILTROS DE QUALIDADE - {tipo_tabela.upper()}")
    print("="*80)
    
    df_original = df
    
    # Filtro 1: Remover zeros e nulos dos saldos
    if tipo_tabela in ['saldos', 'balanco', 'dre']:
        if tipo_tabela == 'saldos':
            campos = ['saldo_final_contabil', 'saldo_inicial_contabil', 
                     'valor_debito_periodo', 'valor_credito_periodo']
        else:
            # Para BP e DRE, verificar campos de valor
            campos = [c for c in df.columns if 'valor' in c.lower() or 'saldo' in c.lower()]
        
        if campos:
            df = filtrar_dados_validos(df, campos)
    
    # Filtro 2: Remover contas inválidas
    if 'cd_conta' in df.columns or 'nm_conta' in df.columns:
        df = filtrar_contas_validas(df)
    
    # Filtro 3: Remover registros duplicados (se houver)
    if 'id_ecd' in df.columns and 'cd_conta' in df.columns:
        colunas_chave = ['id_ecd', 'cd_conta']
        if 'dt_referencia' in df.columns or 'ano_referencia' in df.columns:
            colunas_chave.append('dt_referencia' if 'dt_referencia' in df.columns else 'ano_referencia')
        
        qtd_antes = df.count()
        df = df.dropDuplicates(colunas_chave)
        qtd_depois = df.count()
        
        if qtd_antes != qtd_depois:
            print(f"\n🗑️  Duplicatas removidas: {qtd_antes - qtd_depois:,}")
    
    # Resumo final
    print("\n" + "="*80)
    print("📊 RESUMO DO FILTRO:")
    print("="*80)
    print(f"Registros originais: {df_original.count():,}")
    print(f"Registros filtrados: {df.count():,}")
    print(f"Registros removidos: {df_original.count() - df.count():,}")
    print(f"Taxa de retenção: {(df.count() / df_original.count() * 100):.2f}%")
    print("="*80)
    
    return df


# ============================================================================
# EXEMPLO DE USO - CARREGAR E FILTRAR DADOS
# ============================================================================

def carregar_dados_ecd_filtrados():
    """
    Carrega e filtra os dados principais do ECD.
    USAR ESTA FUNÇÃO NO LUGAR DO CARREGAMENTO DIRETO DAS TABELAS.
    """
    
    print("\n🔄 Carregando dados do ECD com filtros aplicados...")
    
    # 1. Carregar tabela de saldos
    print("\n📂 Carregando: teste.ecd_saldos_contas_v2")
    df_saldos_raw = spark.table('teste.ecd_saldos_contas_v2')
    df_saldos = aplicar_filtros_globais(df_saldos_raw, tipo_tabela='saldos')
    
    # 2. Carregar tabela de empresas (não precisa filtrar saldos)
    print("\n📂 Carregando: teste.ecd_empresas_cadastro")
    df_empresas = spark.table('teste.ecd_empresas_cadastro')
    print(f"📊 Empresas carregadas: {df_empresas.count():,}")
    
    # 3. Carregar tabela ML (não precisa filtrar saldos)
    print("\n📂 Carregando: teste.ecd_ml_empresas_resumo")
    df_ml = spark.table('teste.ecd_ml_empresas_resumo')
    print(f"📊 Registros ML carregados: {df_ml.count():,}")
    
    print("\n✅ Todos os dados carregados e filtrados com sucesso!")
    
    return df_saldos, df_empresas, df_ml


# ============================================================================
# INSTRUÇÕES DE USO NO NOTEBOOK
# ============================================================================

"""
COMO USAR:

1. ADICIONE ESTA CÉLULA LOGO APÓS A CÉLULA DE CONFIGURAÇÃO INICIAL

4. PARA OUTROS TIPOS DE TABELAS:

   df_bp = aplicar_filtros_globais(df_bp_raw, tipo_tabela='balanco')
   df_dre = aplicar_filtros_globais(df_dre_raw, tipo_tabela='dre')
   df_plano = aplicar_filtros_globais(df_plano_raw, tipo_tabela='plano_contas')

================================================================================
"""

print("✅ Funções de filtro carregadas com sucesso!")
print("\n📖 Próximos passos:")
print("   1. Use: df_saldos, df_empresas, df_ml = carregar_dados_ecd_filtrados()")
print("   2. Ou: df_filtrado = aplicar_filtros_globais(df_raw, tipo_tabela='saldos')")

✅ Funções de filtro carregadas com sucesso!

📖 Próximos passos:
   1. Use: df_saldos, df_empresas, df_ml = carregar_dados_ecd_filtrados()
   2. Ou: df_filtrado = aplicar_filtros_globais(df_raw, tipo_tabela='saldos')


In [8]:
# Executar pipeline completo
resultado = main(spark)

# Acessar resultados
df_final = resultado['df_final']
estatisticas = resultado['estatisticas']

# Carregar etapa intermediária
df = carregar_resultado_intermediario(spark, 'dicionarios')

# ============================================================================
# ANÁLISE DOS RESULTADOS (SEM REEXECUTAR O PIPELINE)
# ============================================================================

if resultado:
    print("\n" + "="*80)
    print("📊 ANÁLISE DOS RESULTADOS")
    print("="*80)
    
    # Já temos df_final e estatisticas carregados!
    
    # 1. Estatísticas gerais
    print(f"\n✅ Total de contas: {estatisticas['total']:,}")
    print(f"✅ Classificadas Nível 1: {estatisticas['classificadas_n1']:,} ({estatisticas['classificadas_n1']/estatisticas['total']*100:.1f}%)")
    print(f"✅ Classificadas Nível 2: {estatisticas['classificadas_n2']:,} ({estatisticas['classificadas_n2']/estatisticas['total']*100:.1f}%)")
    print(f"✅ Duração: {estatisticas['duracao']}")
    
    # 2. Distribuição de classificações
    print("\n📊 Distribuição de Classificações Nível 1:")
    df_final.groupBy('classificacao_nivel_1_desc').count() \
        .orderBy('classificacao_nivel_1_desc').show()
    
    # 3. Top 10 empresas por número de contas
    print("\n📊 Top 10 Empresas (por número de contas):")
    df_final.groupBy('id_ecd').count() \
        .orderBy(desc('count')).show(10)
    
    # 4. Métodos de classificação
    print("\n📊 Distribuição por Método de Classificação:")
    df_final.filter(col('metodo_classificacao').isNotNull()) \
        .groupBy('metodo_classificacao').count() \
        .orderBy(desc('count')).show()
    
    # 5. Contas não classificadas
    nao_class = df_final.filter(col('classificacao_nivel_1').isNull()).count()
    print(f"\n⚠️ Contas não classificadas: {nao_class:,} ({nao_class/estatisticas['total']*100:.1f}%)")
    
    # 6. Verificar arquivo salvo
    print(f"\n📂 Resultado salvo em: {PipelineConfig.PATH_FINAL}")
    df_salvo = spark.read.parquet(PipelineConfig.PATH_FINAL)
    print(f"✅ Registros no arquivo: {df_salvo.count():,}")
    print(f"✅ Colunas: {len(df_salvo.columns)}")
    
    # 7. Ver distribuição por origem (do DataFrame intermediário 'df')
    if 'origem' in df.columns:
        print("\n📊 Distribuição por origem (dados intermediários):")
        df.groupBy('origem').count().orderBy('origem').show()
    
    print("\n" + "="*80)
    print("✅ ANÁLISE CONCLUÍDA!")
    print("="*80)
    
else:
    print("\n❌ Resultado não disponível")


🚀 INICIANDO PIPELINE COMPLETO DE CLASSIFICAÇÃO ECD
Data/Hora: 2025-11-13 19:05:37
Ano de referência: 2024

📥 ETAPA 1: CARREGAMENTO DE DADOS

📊 Carregando dados do Balanço Patrimonial e DRE...


  ✅ Balanço Patrimonial: 144,602,899 registros


  ✅ DRE: 40,260,048 registros


  ✅ Total após união: 184,862,947 registros

🔄 Removendo duplicatas...


  ✅ Total após deduplicação: 102,628,360 registros

📊 Distribuição por origem:


...

📥 Carregando resultado intermediário de: SAT_BIG_DATA/data-pipeline/batch/poc/tsevero/dicionarios
✅ Carregado: 99,773,786 registros

📊 ANÁLISE DOS RESULTADOS

✅ Total de contas: 99,773,786
✅ Classificadas Nível 1: 45,804,684 (45.9%)
✅ Classificadas Nível 2: 45,804,684 (45.9%)
✅ Duração: 0:26:28.505710

📊 Distribuição de Classificações Nível 1:


+--------------------------+--------+
|classificacao_nivel_1_desc|   count|
+--------------------------+--------+
|                      NULL|53969102|
|                     ATIVO|24308453|
|                   PASSIVO|14421108|
|                 RESULTADO| 7075123|
+--------------------------+--------+


📊 Top 10 Empresas (por número de contas):


+------+------+
|id_ecd| count|
+------+------+
|432550|434099|
|217595|336081|
|217594|331894|
|487536|330477|
|487521|330474|
|217593|326588|
|217597|261948|
|392704|221321|
|259811|214478|
|344332|179898|
+------+------+
only showing top 10 rows


📊 Distribuição por Método de Classificação:


+--------------------+--------+
|metodo_classificacao|   count|
+--------------------+--------+
|          ESTRUTURAL|45316501|
|          DICIONARIO|  488183|
+--------------------+--------+




⚠️ Contas não classificadas: 53,969,102 (54.1%)

📂 Resultado salvo em: SAT_BIG_DATA/data-pipeline/batch/poc/tsevero/final
✅ Registros no arquivo: 99,773,786
✅ Colunas: 27

📊 Distribuição por origem (dados intermediários):
+------+--------+
|origem|   count|
+------+--------+
|    BP|77910062|
|   DRE|21863724|
+------+--------+


✅ ANÁLISE CONCLUÍDA!


In [9]:
"""
================================================================================
SCRIPT DE SALVAMENTO - RESULTADOS CLASSIFICAÇÃO ECD
================================================================================
Salva resultados do pipeline em múltiplas tabelas Hive otimizadas para análise
e visualização no Streamlit.

Tabelas criadas:
1. teste.ecd_ml_contas_classificadas       - Classificação detalhada de contas
2. teste.ecd_ml_empresas_resumo            - Resumo por empresa
3. teste.ecd_ml_validacao_equacao          - Validação da equação contábil
4. teste.ecd_ml_distribuicao_classes       - Distribuição de classes contábeis
5. teste.ecd_ml_contas_nao_classificadas   - Análise de contas não classificadas

Autor: Sistema de Classificação ECD - SEFAZ/SC
Data: 2025-11-13
================================================================================
"""

from pyspark.sql.functions import (
    col, when, count, countDistinct, sum as spark_sum, avg, max as spark_max,
    min as spark_min, round as spark_round, desc, lit, concat_ws, 
    current_timestamp, abs as spark_abs
)
from datetime import datetime

# ============================================================================
# CONFIGURAÇÃO
# ============================================================================

SCHEMA_DESTINO = "teste"
PREFIXO_TABELAS = "ecd_ml"

# Caminhos dos resultados do pipeline
PATH_FINAL = "SAT_BIG_DATA/data-pipeline/batch/poc/tsevero/final"
PATH_VALIDACAO = "SAT_BIG_DATA/data-pipeline/batch/poc/tsevero/validacao/equacao_contabil"


# ============================================================================
# FUNÇÃO 1: SALVAR CONTAS CLASSIFICADAS (DETALHE)
# ============================================================================

def salvar_contas_classificadas(spark, df_final):
    """
    Tabela principal com classificação detalhada de todas as contas
    """
    
    print("\n" + "="*80)
    print("📊 TABELA 1: ecd_ml_contas_classificadas")
    print("="*80)
    
    nome_tabela = f"{SCHEMA_DESTINO}.{PREFIXO_TABELAS}_contas_classificadas"
    
    # Preparar dados
    df_output = df_final.select(
        # Identificação
        col('id_ecd'),
        col('dt_referencia').alias('ano_mes_referencia'),
        (col('dt_referencia') / 100).cast('int').alias('ano_referencia'),
        
        # Conta
        col('cd_conta_anl').alias('codigo_conta'),
        col('descr_cod_agl').alias('nome_conta'),
        col('cd_conta_sint').alias('codigo_conta_sintetica'),
        col('cd_natureza').alias('natureza_contabil'),
        col('nivel').alias('nivel_hierarquico'),
        col('tp_conta').alias('tipo_conta'),
        
        # Features extraídas
        col('primeiro_digito'),
        col('prefixo_2_chars'),
        col('tem_ponto'),
        col('comprimento_codigo'),
        col('qtd_pontos'),
        col('tem_letra'),
        col('padrao_estrutural'),
        
        # Classificação
        col('classificacao_nivel_1').alias('classe_nivel_1'),
        col('classificacao_nivel_1_desc').alias('classe_nivel_1_descricao'),
        col('classificacao_nivel_2').alias('classe_nivel_2'),
        col('classificacao_final'),
        col('metodo_classificacao'),
        col('confianca_final').alias('confianca_classificacao'),
        
        # Faixa de confiança
        when(col('confianca_final') >= 0.95, 'MUITO_ALTA')
        .when(col('confianca_final') >= 0.90, 'ALTA')
        .when(col('confianca_final') >= 0.80, 'BOA')
        .when(col('confianca_final') >= 0.70, 'MODERADA')
        .otherwise('BAIXA')
        .alias('faixa_confianca'),
        
        # Flag de classificação
        when(col('classificacao_nivel_1').isNotNull(), 1).otherwise(0).alias('flag_classificada'),
        
        # Metadados
        current_timestamp().alias('data_processamento')
    )
    
    # Dropar tabela se existir
    print(f"\n🗑️  Removendo tabela antiga (se existir)...")
    spark.sql(f"DROP TABLE IF EXISTS {nome_tabela} PURGE")
    
    # Criar tabela particionada por ano
    print(f"📝 Criando tabela {nome_tabela}...")
    df_output.write \
        .mode('overwrite') \
        .partitionBy('ano_referencia') \
        .format('parquet') \
        .saveAsTable(nome_tabela)
    
    # Estatísticas
    total = df_output.count()
    classificadas = df_output.filter(col('flag_classificada') == 1).count()
    
    print(f"✅ Tabela criada com sucesso!")
    print(f"   Total de registros: {total:,}")
    print(f"   Classificadas: {classificadas:,} ({classificadas/total*100:.1f}%)")
    
    # Mostrar sample
    print("\n📋 Sample dos dados:")
    spark.sql(f"SELECT * FROM {nome_tabela} LIMIT 5").show(truncate=False)
    
    return nome_tabela


# ============================================================================
# FUNÇÃO 2: SALVAR RESUMO POR EMPRESA
# ============================================================================

def salvar_empresas_resumo(spark, df_final):
    """
    Resumo agregado por empresa com estatísticas de classificação
    """
    
    print("\n" + "="*80)
    print("📊 TABELA 2: ecd_ml_empresas_resumo")
    print("="*80)
    
    nome_tabela = f"{SCHEMA_DESTINO}.{PREFIXO_TABELAS}_empresas_resumo"
    
    # Agregar por empresa
    df_resumo = df_final.groupBy('id_ecd', 'dt_referencia').agg(
        # Contadores
        count('*').alias('total_contas'),
        countDistinct('cd_conta_anl').alias('contas_distintas'),
        spark_sum(when(col('classificacao_nivel_1').isNotNull(), 1).otherwise(0)).alias('contas_classificadas'),
        spark_sum(when(col('classificacao_nivel_1').isNull(), 1).otherwise(0)).alias('contas_nao_classificadas'),
        
        # Por classe
        spark_sum(when(col('classificacao_nivel_1_desc') == 'ATIVO', 1).otherwise(0)).alias('total_ativo'),
        spark_sum(when(col('classificacao_nivel_1_desc') == 'PASSIVO', 1).otherwise(0)).alias('total_passivo'),
        spark_sum(when(col('classificacao_nivel_1_desc') == 'RESULTADO', 1).otherwise(0)).alias('total_resultado'),
        
        # Por método
        spark_sum(when(col('metodo_classificacao') == 'ESTRUTURAL', 1).otherwise(0)).alias('class_estrutural'),
        spark_sum(when(col('metodo_classificacao') == 'DICIONARIO', 1).otherwise(0)).alias('class_dicionario'),
        
        # Confiança
        avg('confianca_final').alias('confianca_media'),
        spark_min('confianca_final').alias('confianca_minima'),
        spark_max('confianca_final').alias('confianca_maxima'),
        
        # Por faixa de confiança
        spark_sum(when(col('confianca_final') >= 0.90, 1).otherwise(0)).alias('confianca_alta'),
        spark_sum(when((col('confianca_final') >= 0.80) & (col('confianca_final') < 0.90), 1).otherwise(0)).alias('confianca_boa'),
        spark_sum(when((col('confianca_final') >= 0.70) & (col('confianca_final') < 0.80), 1).otherwise(0)).alias('confianca_moderada'),
        spark_sum(when(col('confianca_final') < 0.70, 1).otherwise(0)).alias('confianca_baixa'),
        
        # Padrões estruturais
        countDistinct('padrao_estrutural').alias('qtd_padroes_distintos'),
        countDistinct('primeiro_digito').alias('qtd_digitos_distintos'),
        
        # Níveis hierárquicos
        spark_min('nivel').alias('nivel_minimo'),
        spark_max('nivel').alias('nivel_maximo'),
        avg('nivel').alias('nivel_medio')
    )
    
    # Adicionar campos calculados
    df_resumo = df_resumo.withColumn(
        'taxa_classificacao',
        spark_round((col('contas_classificadas') / col('total_contas')) * 100, 2)
    ).withColumn(
        'taxa_ativo',
        spark_round((col('total_ativo') / col('contas_classificadas')) * 100, 2)
    ).withColumn(
        'taxa_passivo',
        spark_round((col('total_passivo') / col('contas_classificadas')) * 100, 2)
    ).withColumn(
        'taxa_resultado',
        spark_round((col('total_resultado') / col('contas_classificadas')) * 100, 2)
    ).withColumn(
        'ano_referencia',
        (col('dt_referencia') / 100).cast('int')
    ).withColumn(
        'confianca_media',
        spark_round('confianca_media', 4)
    ).withColumn(
        'confianca_minima',
        spark_round('confianca_minima', 4)
    ).withColumn(
        'confianca_maxima',
        spark_round('confianca_maxima', 4)
    ).withColumn(
        'nivel_medio',
        spark_round('nivel_medio', 2)
    ).withColumn(
        # Qualidade geral da classificação
        'qualidade_classificacao',
        when(col('taxa_classificacao') >= 95, 'EXCELENTE')
        .when(col('taxa_classificacao') >= 90, 'OTIMA')
        .when(col('taxa_classificacao') >= 80, 'BOA')
        .when(col('taxa_classificacao') >= 70, 'REGULAR')
        .otherwise('RUIM')
    ).withColumn(
        'data_processamento',
        current_timestamp()
    )
    
    # Dropar e criar
    print(f"\n🗑️  Removendo tabela antiga (se existir)...")
    spark.sql(f"DROP TABLE IF EXISTS {nome_tabela} PURGE")
    
    print(f"📝 Criando tabela {nome_tabela}...")
    df_resumo.write \
        .mode('overwrite') \
        .partitionBy('ano_referencia') \
        .format('parquet') \
        .saveAsTable(nome_tabela)
    
    # Estatísticas
    total_empresas = df_resumo.count()
    media_taxa = df_resumo.select(avg('taxa_classificacao')).collect()[0][0]
    
    print(f"✅ Tabela criada com sucesso!")
    print(f"   Total de empresas: {total_empresas:,}")
    print(f"   Taxa média de classificação: {media_taxa:.1f}%")
    
    # Distribuição de qualidade
    print("\n📊 Distribuição de qualidade:")
    spark.sql(f"""
        SELECT 
            qualidade_classificacao,
            COUNT(*) as empresas,
            ROUND(AVG(taxa_classificacao), 2) as taxa_media
        FROM {nome_tabela}
        GROUP BY qualidade_classificacao
        ORDER BY 
            CASE qualidade_classificacao
                WHEN 'EXCELENTE' THEN 1
                WHEN 'OTIMA' THEN 2
                WHEN 'BOA' THEN 3
                WHEN 'REGULAR' THEN 4
                ELSE 5
            END
    """).show(truncate=False)
    
    return nome_tabela


# ============================================================================
# FUNÇÃO 3: SALVAR VALIDAÇÃO DA EQUAÇÃO CONTÁBIL
# ============================================================================

def salvar_validacao_equacao(spark):
    """
    Resultados da validação da equação contábil
    """
    
    print("\n" + "="*80)
    print("📊 TABELA 3: ecd_ml_validacao_equacao")
    print("="*80)
    
    nome_tabela = f"{SCHEMA_DESTINO}.{PREFIXO_TABELAS}_validacao_equacao"
    
    # Carregar validação
    try:
        df_validacao = spark.read.parquet(PATH_VALIDACAO)
        
        # Adicionar campos calculados
        df_output = df_validacao.withColumn(
            'ano_referencia',
            (col('dt_referencia') / 100).cast('int')
        ).withColumn(
            'percentual_diferenca',
            spark_round(
                (col('diferenca') / (spark_abs(col('total_ativo')) + 0.01)) * 100, 
                4
            )
        ).withColumn(
            'status_equacao',
            when(col('equacao_valida'), 'VALIDA').otherwise('INVALIDA')
        ).withColumn(
            'gravidade',
            when(col('diferenca') <= 100, 'BAIXA')
            .when(col('diferenca') <= 1000, 'MEDIA')
            .when(col('diferenca') <= 10000, 'ALTA')
            .otherwise('CRITICA')
        ).withColumn(
            'data_processamento',
            current_timestamp()
        )
        
        # Selecionar colunas
        df_output = df_output.select(
            'id_ecd',
            'dt_referencia',
            'ano_referencia',
            'total_ativo',
            'total_passivo',
            'total_pl',
            'passivo_pl_calculado',
            'diferenca',
            'percentual_diferenca',
            'equacao_valida',
            'status_equacao',
            'gravidade',
            'data_processamento'
        )
        
        # Dropar e criar
        print(f"\n🗑️  Removendo tabela antiga (se existir)...")
        spark.sql(f"DROP TABLE IF EXISTS {nome_tabela} PURGE")
        
        print(f"📝 Criando tabela {nome_tabela}...")
        df_output.write \
            .mode('overwrite') \
            .partitionBy('ano_referencia') \
            .format('parquet') \
            .saveAsTable(nome_tabela)
        
        # Estatísticas
        total = df_output.count()
        validas = df_output.filter(col('equacao_valida')).count()
        
        print(f"✅ Tabela criada com sucesso!")
        print(f"   Total de empresas: {total:,}")
        print(f"   Equação válida: {validas:,} ({validas/total*100:.1f}%)")
        
        # Distribuição por gravidade
        print("\n📊 Distribuição por gravidade (inválidas):")
        spark.sql(f"""
            SELECT 
                gravidade,
                COUNT(*) as empresas,
                ROUND(AVG(diferenca), 2) as diferenca_media,
                ROUND(AVG(percentual_diferenca), 2) as percentual_medio
            FROM {nome_tabela}
            WHERE NOT equacao_valida
            GROUP BY gravidade
            ORDER BY 
                CASE gravidade
                    WHEN 'CRITICA' THEN 1
                    WHEN 'ALTA' THEN 2
                    WHEN 'MEDIA' THEN 3
                    ELSE 4
                END
        """).show(truncate=False)
        
        return nome_tabela
        
    except Exception as e:
        print(f"⚠️  Erro ao processar validação: {str(e)}")
        return None


# ============================================================================
# FUNÇÃO 4: SALVAR DISTRIBUIÇÃO DE CLASSES
# ============================================================================

def salvar_distribuicao_classes(spark, df_final):
    """
    Distribuição de classes contábeis para análise visual
    """
    
    print("\n" + "="*80)
    print("📊 TABELA 4: ecd_ml_distribuicao_classes")
    print("="*80)
    
    nome_tabela = f"{SCHEMA_DESTINO}.{PREFIXO_TABELAS}_distribuicao_classes"
    
    # Agregar por empresa e classe
    df_dist = df_final.filter(col('classificacao_nivel_1').isNotNull()) \
        .groupBy(
            'id_ecd',
            'dt_referencia',
            'classificacao_nivel_1_desc',
            'classificacao_nivel_2'
        ).agg(
            count('*').alias('quantidade_contas'),
            avg('confianca_final').alias('confianca_media'),
            countDistinct('padrao_estrutural').alias('qtd_padroes'),
            countDistinct('primeiro_digito').alias('qtd_digitos')
        )
    
    # Adicionar ano e percentuais
    df_dist = df_dist.withColumn(
        'ano_referencia',
        (col('dt_referencia') / 100).cast('int')
    ).withColumn(
        'confianca_media',
        spark_round('confianca_media', 4)
    ).withColumn(
        'data_processamento',
        current_timestamp()
    )
    
    # Dropar e criar
    print(f"\n🗑️  Removendo tabela antiga (se existir)...")
    spark.sql(f"DROP TABLE IF EXISTS {nome_tabela} PURGE")
    
    print(f"📝 Criando tabela {nome_tabela}...")
    df_dist.write \
        .mode('overwrite') \
        .partitionBy('ano_referencia', 'classificacao_nivel_1_desc') \
        .format('parquet') \
        .saveAsTable(nome_tabela)
    
    print(f"✅ Tabela criada com sucesso!")
    print(f"   Total de registros: {df_dist.count():,}")
    
    # Top 10 classificações
    print("\n📊 Top 10 combinações mais frequentes:")
    spark.sql(f"""
        SELECT 
            classificacao_nivel_1_desc,
            classificacao_nivel_2,
            SUM(quantidade_contas) as total_contas,
            ROUND(AVG(confianca_media), 3) as confianca
        FROM {nome_tabela}
        GROUP BY classificacao_nivel_1_desc, classificacao_nivel_2
        ORDER BY total_contas DESC
        LIMIT 10
    """).show(truncate=False)
    
    return nome_tabela


# ============================================================================
# FUNÇÃO 5: SALVAR CONTAS NÃO CLASSIFICADAS
# ============================================================================

def salvar_contas_nao_classificadas(spark, df_final):
    """
    Análise detalhada das contas que não foram classificadas
    """
    
    print("\n" + "="*80)
    print("📊 TABELA 5: ecd_ml_contas_nao_classificadas")
    print("="*80)
    
    nome_tabela = f"{SCHEMA_DESTINO}.{PREFIXO_TABELAS}_contas_nao_classificadas"
    
    # Filtrar não classificadas
    df_nao_class = df_final.filter(col('classificacao_nivel_1').isNull())
    
    # Agregar por padrão
    df_analise = df_nao_class.groupBy(
        'dt_referencia',
        'primeiro_digito',
        'padrao_estrutural',
        'cd_natureza',
        'tem_ponto',
        'tem_letra'
    ).agg(
        count('*').alias('quantidade'),
        countDistinct('id_ecd').alias('empresas_afetadas'),
        avg('comprimento_codigo').alias('comprimento_medio'),
        avg('qtd_pontos').alias('pontos_medio')
    )
    
    # Adicionar campos
    df_analise = df_analise.withColumn(
        'ano_referencia',
        (col('dt_referencia') / 100).cast('int')
    ).withColumn(
        'comprimento_medio',
        spark_round('comprimento_medio', 2)
    ).withColumn(
        'pontos_medio',
        spark_round('pontos_medio', 2)
    ).withColumn(
        'tipo_problema',
        when(col('primeiro_digito').isin(['0', '4', '5', '6', '7', '8', '9']), 'CODIGO_NAO_PADRONIZADO')
        .when(col('tem_letra') == 1, 'CODIGO_COM_LETRA')
        .otherwise('OUTRO')
    ).withColumn(
        'data_processamento',
        current_timestamp()
    )
    
    # Dropar e criar
    print(f"\n🗑️  Removendo tabela antiga (se existir)...")
    spark.sql(f"DROP TABLE IF EXISTS {nome_tabela} PURGE")
    
    print(f"📝 Criando tabela {nome_tabela}...")
    df_analise.write \
        .mode('overwrite') \
        .partitionBy('ano_referencia') \
        .format('parquet') \
        .saveAsTable(nome_tabela)
    
    total_nao_class = df_nao_class.count()
    print(f"✅ Tabela criada com sucesso!")
    print(f"   Total de contas não classificadas: {total_nao_class:,}")
    
    # Distribuição por tipo
    print("\n📊 Distribuição por tipo de problema:")
    spark.sql(f"""
        SELECT 
            tipo_problema,
            SUM(quantidade) as total_contas,
            SUM(empresas_afetadas) as total_empresas
        FROM {nome_tabela}
        GROUP BY tipo_problema
        ORDER BY total_contas DESC
    """).show(truncate=False)
    
    # Top padrões problemáticos
    print("\n📊 Top 10 padrões não classificados:")
    spark.sql(f"""
        SELECT 
            primeiro_digito,
            padrao_estrutural,
            cd_natureza,
            SUM(quantidade) as total
        FROM {nome_tabela}
        GROUP BY primeiro_digito, padrao_estrutural, cd_natureza
        ORDER BY total DESC
        LIMIT 10
    """).show(truncate=False)
    
    return nome_tabela


# ============================================================================
# FUNÇÃO PRINCIPAL: SALVAR TODOS OS RESULTADOS
# ============================================================================

def salvar_todos_resultados(spark):
    """
    Salva todos os resultados em tabelas Hive
    """
    
    print("\n" + "="*80)
    print("💾 SALVAMENTO DE RESULTADOS EM TABELAS HIVE")
    print("="*80)
    print(f"Schema: {SCHEMA_DESTINO}")
    print(f"Prefixo: {PREFIXO_TABELAS}_*")
    print(f"Data: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("="*80)
    
    inicio = datetime.now()
    tabelas_criadas = []
    
    try:
        # Carregar resultado final
        print(f"\n📥 Carregando resultados do pipeline...")
        print(f"   Path: {PATH_FINAL}")
        df_final = spark.read.parquet(PATH_FINAL)
        print(f"✅ Carregado: {df_final.count():,} registros")
        
        # 1. Contas classificadas (detalhe)
        tabela = salvar_contas_classificadas(spark, df_final)
        tabelas_criadas.append(tabela)
        
        # 2. Resumo por empresa
        tabela = salvar_empresas_resumo(spark, df_final)
        tabelas_criadas.append(tabela)
        
        # 3. Validação equação
        tabela = salvar_validacao_equacao(spark)
        if tabela:
            tabelas_criadas.append(tabela)
        
        # 4. Distribuição de classes
        tabela = salvar_distribuicao_classes(spark, df_final)
        tabelas_criadas.append(tabela)
        
        # 5. Contas não classificadas
        tabela = salvar_contas_nao_classificadas(spark, df_final)
        tabelas_criadas.append(tabela)
        
        # Relatório final
        fim = datetime.now()
        duracao = fim - inicio
        
        print("\n" + "="*80)
        print("✅ SALVAMENTO CONCLUÍDO COM SUCESSO!")
        print("="*80)
        print(f"Tempo total: {duracao}")
        print(f"\n📊 TABELAS CRIADAS ({len(tabelas_criadas)}):")
        for i, tabela in enumerate(tabelas_criadas, 1):
            print(f"   {i}. {tabela}")
        
        print("\n" + "="*80)
        print("🎯 PRÓXIMOS PASSOS:")
        print("="*80)
        print("""
        1. Remover LIMITE_REGISTROS para processar população completa:
           PipelineConfig.LIMITE_REGISTROS = None
        
        2. Executar pipeline completo (945M registros, 73.335 empresas)
        
        3. Visualizar no Streamlit usando as tabelas:
           - teste.ecd_ml_contas_classificadas
           - teste.ecd_ml_empresas_resumo
           - teste.ecd_ml_validacao_equacao
           - teste.ecd_ml_distribuicao_classes
           - teste.ecd_ml_contas_nao_classificadas
        
        4. Queries úteis para Streamlit:
        
           # Empresas com melhor classificação
           SELECT * FROM teste.ecd_ml_empresas_resumo 
           WHERE qualidade_classificacao = 'EXCELENTE'
           ORDER BY taxa_classificacao DESC
           
           # Distribuição de classes
           SELECT 
               classificacao_nivel_1_desc,
               SUM(quantidade_contas) as total
           FROM teste.ecd_ml_distribuicao_classes
           GROUP BY classificacao_nivel_1_desc
           
           # Empresas com equação inválida
           SELECT * FROM teste.ecd_ml_validacao_equacao
           WHERE NOT equacao_valida
           ORDER BY diferenca DESC
        """)
        
        return tabelas_criadas
        
    except Exception as e:
        print(f"\n❌ ERRO: {str(e)}")
        import traceback
        traceback.print_exc()
        return None


# ============================================================================
# FUNÇÃO AUXILIAR: ESTATÍSTICAS DAS TABELAS
# ============================================================================

def mostrar_estatisticas_tabelas(spark):
    """
    Mostra estatísticas das tabelas criadas
    """
    
    print("\n" + "="*80)
    print("📊 ESTATÍSTICAS DAS TABELAS")
    print("="*80)
    
    tabelas = [
        'ecd_ml_contas_classificadas',
        'ecd_ml_empresas_resumo',
        'ecd_ml_validacao_equacao',
        'ecd_ml_distribuicao_classes',
        'ecd_ml_contas_nao_classificadas'
    ]
    
    for tabela in tabelas:
        nome_completo = f"{SCHEMA_DESTINO}.{tabela}"
        try:
            df = spark.table(nome_completo)
            total = df.count()
            colunas = len(df.columns)
            
            print(f"\n📋 {tabela}")
            print(f"   Registros: {total:,}")
            print(f"   Colunas: {colunas}")
            print(f"   Partições: {df.rdd.getNumPartitions()}")
            
            # Mostrar schema resumido
            print(f"   Schema: {', '.join(df.columns[:10])}...")
            
        except Exception as e:
            print(f"\n⚠️  {tabela}: {str(e)}")


# ============================================================================
# EXECUÇÃO
# ============================================================================

if __name__ == "__main__":
    print("""
    ================================================================================
    SCRIPT DE SALVAMENTO - RESULTADOS CLASSIFICAÇÃO ECD
    ================================================================================
    
    Para executar:
    
    1. Após executar o pipeline:
       >>> exec(open('/mnt/user-data/outputs/salvar_resultados_hive.py').read())
       >>> tabelas = salvar_todos_resultados(spark)
    
    2. Ver estatísticas:
       >>> mostrar_estatisticas_tabelas(spark)
    
    3. Consultar tabelas no Streamlit:
       >>> df = spark.table('teste.ecd_ml_empresas_resumo')
       >>> df.show()
    
    ================================================================================
    """)


    SCRIPT DE SALVAMENTO - RESULTADOS CLASSIFICAÇÃO ECD
    
    Para executar:
    
    1. Após executar o pipeline:
       >>> exec(open('/mnt/user-data/outputs/salvar_resultados_hive.py').read())
       >>> tabelas = salvar_todos_resultados(spark)
    
    2. Ver estatísticas:
       >>> mostrar_estatisticas_tabelas(spark)
    
    3. Consultar tabelas no Streamlit:
       >>> df = spark.table('teste.ecd_ml_empresas_resumo')
       >>> df.show()
    
    


In [10]:
# 4. Salvar em tabelas Hive
tabelas = salvar_todos_resultados(spark)

# 5. Validar Ver Estatisticas
mostrar_estatisticas_tabelas(spark)
    
# 6 Consultar tabelas no Streamlit:
df = spark.table('teste.ecd_ml_empresas_resumo')
df.show()


💾 SALVAMENTO DE RESULTADOS EM TABELAS HIVE
Schema: teste
Prefixo: ecd_ml_*
Data: 2025-11-13 19:38:12

📥 Carregando resultados do pipeline...
   Path: SAT_BIG_DATA/data-pipeline/batch/poc/tsevero/final
✅ Carregado: 99,773,786 registros

📊 TABELA 1: ecd_ml_contas_classificadas

🗑️  Removendo tabela antiga (se existir)...
📝 Criando tabela teste.ecd_ml_contas_classificadas...


✅ Tabela criada com sucesso!
   Total de registros: 99,773,786
   Classificadas: 45,804,684 (45.9%)

📋 Sample dos dados:
+------+------------------+------------------+----------------------------+----------------------+-----------------+-----------------+----------+---------------+---------------+---------+------------------+----------+---------+-----------------+--------------+------------------------+--------------+-------------------+--------------------+-----------------------+---------------+-----------------+--------------------------+--------------+
|id_ecd|ano_mes_referencia|codigo_conta      |nome_conta                  |codigo_conta_sintetica|natureza_contabil|nivel_hierarquico|tipo_conta|primeiro_digito|prefixo_2_chars|tem_ponto|comprimento_codigo|qtd_pontos|tem_letra|padrao_estrutural|classe_nivel_1|classe_nivel_1_descricao|classe_nivel_2|classificacao_final|metodo_classificacao|confianca_classificacao|faixa_confianca|flag_classificada|data_processamento        |ano_referen

✅ Tabela criada com sucesso!
   Total de empresas: 384,515
   Taxa média de classificação: 50.3%

📊 Distribuição de qualidade:
+-----------------------+--------+----------+
|qualidade_classificacao|empresas|taxa_media|
+-----------------------+--------+----------+
|EXCELENTE              |28800   |99.42     |
|OTIMA                  |9627    |92.32     |
|BOA                    |31127   |84.41     |
|REGULAR                |56836   |74.43     |
|RUIM                   |258125  |33.83     |
+-----------------------+--------+----------+


📊 TABELA 3: ecd_ml_validacao_equacao

🗑️  Removendo tabela antiga (se existir)...
📝 Criando tabela teste.ecd_ml_validacao_equacao...
✅ Tabela criada com sucesso!
   Total de empresas: 379,007
   Equação válida: 82,821 (21.9%)

📊 Distribuição por gravidade (inválidas):
+---------+--------+---------------+----------------+
|gravidade|empresas|diferenca_media|percentual_medio|
+---------+--------+---------------+----------------+
|CRITICA  |276895  |417974

✅ Tabela criada com sucesso!


   Total de registros: 1,206,598

📊 Top 10 combinações mais frequentes:
+--------------------------+---------------------+------------+---------+
|classificacao_nivel_1_desc|classificacao_nivel_2|total_contas|confianca|
+--------------------------+---------------------+------------+---------+
|ATIVO                     |1.0                  |21809053    |0.734    |
|PASSIVO                   |2.0                  |13108673    |0.786    |
|RESULTADO                 |3.0                  |7054066     |0.548    |
|ATIVO                     |1.01                 |1520498     |0.797    |
|PASSIVO                   |2.01                 |876119      |0.797    |
|ATIVO                     |1.02                 |548446      |0.796    |
|ATIVO                     |1.02.01              |348150      |0.75     |
|PASSIVO                   |2.02                 |235869      |0.798    |
|PASSIVO                   |2.03                 |163777      |0.779    |
|ATIVO                     |1.02.03     

✅ Tabela criada com sucesso!
   Total de contas não classificadas: 53,969,102

📊 Distribuição por tipo de problema:
+----------------------+------------+--------------+
|tipo_problema         |total_contas|total_empresas|
+----------------------+------------+--------------+
|CODIGO_COM_LETRA      |33908461    |689231        |
|CODIGO_NAO_PADRONIZADO|20058616    |1516472       |
|OUTRO                 |2025        |166           |
+----------------------+------------+--------------+


📊 Top 10 padrões não classificados:
+---------------+-----------------+-----------+--------+
|primeiro_digito|padrao_estrutural|cd_natureza|total   |
+---------------+-----------------+-----------+--------+
|               |PREFIXO_BP       |D          |15319668|
|               |PREFIXO_BP       |C          |7775744 |
|               |PREFIXO_DRE      |D          |4758185 |
|4              |SOMENTE_NUMEROS  |D          |2230445 |
|               |OUTRO            |C          |2211743 |
|5              |SO


📋 ecd_ml_contas_classificadas
   Registros: 99,773,786
   Colunas: 25
   Partições: 36
   Schema: id_ecd, ano_mes_referencia, codigo_conta, nome_conta, codigo_conta_sintetica, natureza_contabil, nivel_hierarquico, tipo_conta, primeiro_digito, prefixo_2_chars...

📋 ecd_ml_empresas_resumo
   Registros: 384,515
   Colunas: 30
   Partições: 25
   Schema: id_ecd, dt_referencia, total_contas, contas_distintas, contas_classificadas, contas_nao_classificadas, total_ativo, total_passivo, total_resultado, class_estrutural...

📋 ecd_ml_validacao_equacao
   Registros: 379,007
   Colunas: 13
   Partições: 25
   Schema: id_ecd, dt_referencia, total_ativo, total_passivo, total_pl, passivo_pl_calculado, diferenca, percentual_diferenca, equacao_valida, status_equacao...

📋 ecd_ml_distribuicao_classes
   Registros: 1,206,598
   Colunas: 10
   Partições: 28
   Schema: id_ecd, dt_referencia, classificacao_nivel_2, quantidade_contas, confianca_media, qtd_padroes, qtd_digitos, data_processamento, ano_refer

In [11]:
"""
SCRIPT DE VERIFICAÇÃO - IDENTIFICAR TABELAS CORRETAS
=====================================================

Execute este script no seu notebook para identificar as tabelas corretas
e verificar se os campos necessários existem.
"""

from pyspark.sql.functions import *

print("="*80)
print("VERIFICAÇÃO DE TABELAS - SISTEMA ECD HÍBRIDO")
print("="*80)

# ==============================================================================
# 1. VERIFICAR TABELA DE SALDOS
# ==============================================================================
print("\n" + "="*80)
print("1. TABELA DE SALDOS: teste.ecd_saldos_contas_v2")
print("="*80)

try:
    df_saldos, df_empresas, df_ml = carregar_dados_ecd_filtrados()
    
    print("\n✅ Tabela encontrada!")
    print(f"📊 Total de registros: {df_saldos.count():,}")
    
    print("\n📋 Schema da tabela:")
    df_saldos.printSchema()
    
    print("\n🔍 Primeiros 3 registros:")
    df_saldos.limit(3).show(truncate=False)
    
    # Verificar campos essenciais
    colunas = df_saldos.columns
    campos_essenciais = {
        'id_ecd': 'id_ecd' in colunas,
        'dt_referencia': 'dt_referencia' in colunas,
        'saldo (ou vl_saldo)': 'saldo' in colunas or 'vl_saldo' in colunas,
        'classificacao_nivel_1': 'classificacao_nivel_1' in colunas,
        'classificacao_nivel_2': 'classificacao_nivel_2' in colunas,
        'classificacao_nivel_3': 'classificacao_nivel_3' in colunas,
    }
    
    print("\n✅ Campos essenciais:")
    for campo, existe in campos_essenciais.items():
        status = "✅" if existe else "❌"
        print(f"  {status} {campo}")
    
    # Determinar nome correto do campo saldo
    campo_saldo = 'saldo' if 'saldo' in colunas else 'vl_saldo' if 'vl_saldo' in colunas else None
    if campo_saldo:
        print(f"\n💡 Use o campo: '{campo_saldo}' para os saldos")
    
except Exception as e:
    print(f"\n❌ ERRO: {e}")
    print("Verifique se a tabela existe e está acessível")


# ==============================================================================
# 2. VERIFICAR TABELA DE EMPRESAS
# ==============================================================================
print("\n" + "="*80)
print("2. TABELA DE EMPRESAS: teste.ecd_empresas_cadastro")
print("="*80)

try:
    df_empresas = spark.table('teste.ecd_empresas_cadastro')
    
    print("\n✅ Tabela encontrada!")
    print(f"📊 Total de empresas: {df_empresas.count():,}")
    
    print("\n📋 Schema da tabela:")
    df_empresas.printSchema()
    
    print("\n🔍 Primeiros 3 registros:")
    df_empresas.limit(3).show(truncate=False)
    
    # Verificar campos essenciais
    colunas = df_empresas.columns
    campos_essenciais = {
        'id_ecd': 'id_ecd' in colunas,
        'porte_empresa (ou classificacao_porte)': 'porte_empresa' in colunas or 'classificacao_porte' in colunas,
        'setor_atividade (ou cnae)': any(x in colunas for x in ['setor_atividade', 'cnae_principal', 'cnae'])
    }
    
    print("\n✅ Campos essenciais:")
    for campo, existe in campos_essenciais.items():
        status = "✅" if existe else "❌"
        print(f"  {status} {campo}")
    
    # Determinar nomes corretos dos campos
    if 'porte_empresa' in colunas:
        print(f"\n💡 Use o campo: 'porte_empresa'")
    elif 'classificacao_porte' in colunas:
        print(f"\n💡 Use o campo: 'classificacao_porte' (renomear para 'porte_empresa')")
    
    if 'setor_atividade' in colunas:
        print(f"💡 Use o campo: 'setor_atividade'")
    elif 'cnae_principal' in colunas:
        print(f"💡 Use o campo: 'cnae_principal' (renomear para 'setor_atividade')")
    elif 'cnae' in colunas:
        print(f"💡 Use o campo: 'cnae' (renomear para 'setor_atividade')")
    
except Exception as e:
    print(f"\n❌ ERRO: {e}")
    print("Verifique se a tabela existe e está acessível")


# ==============================================================================
# 3. VERIFICAR TABELAS ML DISPONÍVEIS
# ==============================================================================
print("\n" + "="*80)
print("3. TABELAS ML DISPONÍVEIS")
print("="*80)

tabelas_ml = [
    'teste.ecd_ml_contas_classificadas',
    'teste.ecd_ml_contas_nao_classificadas',
    'teste.ecd_ml_distribuicao_classes',
    'teste.ecd_ml_empresas_resumo',
    'teste.ecd_ml_validacao_equacao'
]

tabelas_encontradas = []

for tabela in tabelas_ml:
    try:
        df_temp = spark.table(tabela)
        count = df_temp.count()
        tabelas_encontradas.append(tabela)
        print(f"\n✅ {tabela}")
        print(f"   📊 Registros: {count:,}")
        print(f"   📋 Colunas: {', '.join(df_temp.columns[:10])}")  # Primeiras 10 colunas
    except:
        print(f"\n❌ {tabela} - NÃO ENCONTRADA")


# ==============================================================================
# 4. ANALISAR TABELA ML RESUMO (PRINCIPAL CANDIDATA)
# ==============================================================================
print("\n" + "="*80)
print("4. ANÁLISE DETALHADA: teste.ecd_ml_empresas_resumo")
print("="*80)

if 'teste.ecd_ml_empresas_resumo' in tabelas_encontradas:
    try:
        df_ml_resumo = spark.table('teste.ecd_ml_empresas_resumo')
        
        print("\n✅ Esta é a tabela ideal para df_ml_features!")
        
        print("\n📋 Schema completo:")
        df_ml_resumo.printSchema()
        
        print("\n🔍 Primeiros 3 registros:")
        df_ml_resumo.limit(3).show(truncate=False)
        
        # Verificar campo de confiança
        colunas = df_ml_resumo.columns
        if 'confianca_media' in colunas:
            print("\n✅ Campo 'confianca_media' encontrado - PERFEITO!")
            
            # Estatísticas de confiança
            stats = df_ml_resumo.select('confianca_media').describe()
            print("\n📊 Estatísticas de confiança_media:")
            stats.show()
        else:
            print("\n⚠️  Campo 'confianca_media' NÃO encontrado")
            print("Campos disponíveis que podem servir:")
            for col in colunas:
                if 'confianca' in col.lower() or 'score' in col.lower() or 'media' in col.lower():
                    print(f"  - {col}")
        
        print("\n💡 RECOMENDAÇÃO:")
        print("   Use: df_ml_features = spark.table('teste.ecd_ml_empresas_resumo')")
        
    except Exception as e:
        print(f"\n❌ ERRO ao analisar: {e}")
else:
    print("\n❌ Tabela teste.ecd_ml_empresas_resumo NÃO encontrada")
    print("\n💡 SOLUÇÃO ALTERNATIVA:")
    print("   Criar resumo a partir de teste.ecd_ml_contas_classificadas:")
    print("""
    df_ml_features = spark.table('teste.ecd_ml_contas_classificadas') \\
        .groupBy('id_ecd', 'dt_referencia').agg(
            avg('confianca').alias('confianca_media'),
            count('*').alias('total_contas')
        )
    """)


# ==============================================================================
# 5. CÓDIGO FINAL RECOMENDADO
# ==============================================================================
print("\n" + "="*80)
print("5. CÓDIGO FINAL RECOMENDADO")
print("="*80)

print("✅ Análise híbrida concluída com sucesso!")
""")

print("\n" + "="*80)
print("VERIFICAÇÃO CONCLUÍDA")
print("="*80)

VERIFICAÇÃO DE TABELAS - SISTEMA ECD HÍBRIDO

1. TABELA DE SALDOS: teste.ecd_saldos_contas_v2

🔄 Carregando dados do ECD com filtros aplicados...

📂 Carregando: teste.ecd_saldos_contas_v2
🔍 APLICANDO FILTROS DE QUALIDADE - SALDOS


📊 Registros antes do filtro: 945,499,468


📊 Registros depois do filtro: 929,622,972
🗑️  Registros removidos (zeros/nulos): 15,876,496


✅ Taxa de retenção: 98.32%

📋 Filtrando contas inválidas...


📊 Registros antes: 929,622,972


...

Registros filtrados: 159,534,314


Registros removidos: 785,965,154


Taxa de retenção: 16.87%

📂 Carregando: teste.ecd_empresas_cadastro
📊 Empresas carregadas: 349,301

📂 Carregando: teste.ecd_ml_empresas_resumo
📊 Registros ML carregados: 384,515

✅ Todos os dados carregados e filtrados com sucesso!

✅ Tabela encontrada!


📊 Total de registros: 159,534,314

📋 Schema da tabela:
root
 |-- id_ecd: long (nullable = true)
 |-- cnpj: string (nullable = true)
 |-- ano_referencia: integer (nullable = true)
 |-- data_inicio_periodo: timestamp (nullable = true)
 |-- data_fim_periodo: timestamp (nullable = true)
 |-- cd_conta: string (nullable = true)
 |-- cd_centro_custos: string (nullable = true)
 |-- cd_conta_referencial: string (nullable = true)
 |-- nm_conta: string (nullable = true)
 |-- tipo_conta: string (nullable = true)
 |-- nivel_conta: short (nullable = true)
 |-- descricao_grupo_balanco: string (nullable = true)
 |-- grupo_balanco: string (nullable = true)
 |-- saldo_inicial_abs: double (nullable = true)
 |-- indicador_saldo_inicial: string (nullable = true)
 |-- saldo_inicial_contabil: double (nullable = true)
 |-- valor_debito_periodo: double (nullable = true)
 |-- valor_credito_periodo: double (nullable = true)
 |-- movimentacao_liquida: double (nullable = true)
 |-- saldo_final_abs: double (nullabl

+------+--------------+--------------+-------------------+-------------------+----------------+----------------+--------------------+---------------+----------+-----------+-----------------------+------------------+-----------------+-----------------------+----------------------+--------------------+---------------------+--------------------+---------------+---------------------+--------------------+
|id_ecd|cnpj          |ano_referencia|data_inicio_periodo|data_fim_periodo   |cd_conta        |cd_centro_custos|cd_conta_referencial|nm_conta       |tipo_conta|nivel_conta|descricao_grupo_balanco|grupo_balanco     |saldo_inicial_abs|indicador_saldo_inicial|saldo_inicial_contabil|valor_debito_periodo|valor_credito_periodo|movimentacao_liquida|saldo_final_abs|indicador_saldo_final|saldo_final_contabil|
+------+--------------+--------------+-------------------+-------------------+----------------+----------------+--------------------+---------------+----------+-----------+--------------------

In [12]:
"""
CÓDIGO CORRIGIDO - SISTEMA ECD HÍBRIDO
Adaptado para a estrutura real das tabelas

MAPEAMENTO DE CAMPOS:
- dt_referencia → NÃO EXISTE, usar: ano_referencia + criar período
- saldo → NÃO EXISTE, usar: saldo_final_contabil
- classificacao_nivel_X → NÃO EXISTEM, derivar de: grupo_balanco ou cd_conta_referencial
- cd_conta_plano_contas_ref → cd_conta_referencial
- porte_empresa → empresa_grande_porte
- setor_atividade → cnae_secao_descricao
"""

from pyspark.sql.functions import *
from pyspark.sql.window import Window

# ==============================================================================
# FUNÇÃO 1: CALCULAR INDICADORES ECONÔMICO-FINANCEIROS (CORRIGIDA)
# ==============================================================================

def calcular_indicadores_economico_financeiros(df_saldos, df_empresas):
    """
    Calcula 10 indicadores econômico-financeiros
    VERSÃO CORRIGIDA para campos reais da tabela
    """
    
    print("📊 Calculando indicadores econômico-financeiros...")
    
    # Preparar dados: adicionar dt_referencia e classificações
    df_prep = df_saldos \
        .withColumn('dt_referencia', 
                   concat(col('ano_referencia'), 
                          lpad(month(col('data_fim_periodo')), 2, '0')).cast('int')) \
        .withColumn('saldo', col('saldo_final_contabil')) \
        .withColumn('cd_conta_plano_contas_ref', col('cd_conta_referencial'))
    
    # Derivar classificações a partir do grupo_balanco e cd_conta_referencial
    df_prep = df_prep \
        .withColumn('classificacao_nivel_1',
                   when(col('grupo_balanco').rlike('^Ativo'), '1')
                   .when(col('grupo_balanco').rlike('^Passivo'), '2')
                   .when(col('cd_conta_referencial').rlike('^3\\.'), '3')
                   .otherwise(substring(col('cd_conta_referencial'), 1, 1))) \
        .withColumn('classificacao_nivel_2',
                   when(col('grupo_balanco') == 'Ativo Circulante', '1.01')
                   .when(col('grupo_balanco').rlike('Ativo.*Longo'), '1.02')
                   .when(col('grupo_balanco').rlike('Ativo.*Permanente|Imobilizado|Intang'), '1.03')
                   .when(col('grupo_balanco') == 'Passivo Circulante', '2.01')
                   .when(col('grupo_balanco').rlike('Passivo.*Longo'), '2.02')
                   .when(col('grupo_balanco').rlike('Patrim|PL'), '2.03')
                   .otherwise(substring(col('cd_conta_referencial'), 1, 4)))
    
    # Agregar saldos por empresa e período
    df_componentes = df_prep.groupBy('id_ecd', 'dt_referencia').agg(
        # ATIVO
        sum(when(col('classificacao_nivel_2') == '1.01', col('saldo')).otherwise(0)).alias('ativo_circulante'),
        sum(when(col('classificacao_nivel_2') == '1.02', col('saldo')).otherwise(0)).alias('realizavel_longo_prazo'),
        sum(when(col('classificacao_nivel_2') == '1.03', col('saldo')).otherwise(0)).alias('ativo_permanente'),
        sum(when(col('classificacao_nivel_1') == '1', col('saldo')).otherwise(0)).alias('ativo_total'),
        
        # PASSIVO
        sum(when(col('classificacao_nivel_2') == '2.01', col('saldo')).otherwise(0)).alias('passivo_circulante'),
        sum(when(col('classificacao_nivel_2') == '2.02', col('saldo')).otherwise(0)).alias('exigivel_longo_prazo'),
        sum(when(col('classificacao_nivel_2') == '2.03', col('saldo')).otherwise(0)).alias('patrimonio_liquido'),
        
        # DRE - Usar cd_conta_referencial para identificar
        sum(when(col('cd_conta_referencial').rlike('^3\\.01'), col('saldo')).otherwise(0)).alias('receita_bruta'),
        sum(when(col('cd_conta_referencial').rlike('^3\\.02'), col('saldo')).otherwise(0)).alias('deducoes_receita'),
        sum(when(col('cd_conta_referencial').rlike('^3\\.03'), col('saldo')).otherwise(0)).alias('receita_liquida'),
        sum(when(col('cd_conta_referencial').rlike('^3\\.09'), col('saldo')).otherwise(0)).alias('lucro_liquido'),
        sum(when(col('cd_conta_referencial').rlike('^3\\.04'), col('saldo')).otherwise(0)).alias('custo_mercadorias')
    )
    
    # Calcular índices
    df_indicadores = df_componentes \
        .withColumn('exigivel_total', col('passivo_circulante') + col('exigivel_longo_prazo')) \
        .withColumn('recursos_nao_correntes', col('patrimonio_liquido') + col('exigivel_longo_prazo')) \
        \
        .withColumn('liquidez_corrente', 
                   when(col('passivo_circulante') != 0, 
                        col('ativo_circulante') / col('passivo_circulante')).otherwise(None)) \
        \
        .withColumn('liquidez_geral',
                   when(col('exigivel_total') != 0,
                        (col('ativo_circulante') + col('realizavel_longo_prazo')) / col('exigivel_total')).otherwise(None)) \
        \
        .withColumn('endividamento',
                   when(col('ativo_total') != 0,
                        col('exigivel_total') / col('ativo_total')).otherwise(None)) \
        \
        .withColumn('participacao_capital_proprio',
                   when(col('exigivel_total') != 0,
                        col('patrimonio_liquido') / col('exigivel_total')).otherwise(None)) \
        \
        .withColumn('giro_ativo',
                   when(col('ativo_total') != 0,
                        col('receita_liquida') / col('ativo_total')).otherwise(None)) \
        \
        .withColumn('margem_liquida',
                   when(col('receita_liquida') != 0,
                        col('lucro_liquido') / col('receita_liquida')).otherwise(None)) \
        \
        .withColumn('retorno_ativo',
                   when(col('ativo_total') != 0,
                        col('lucro_liquido') / col('ativo_total')).otherwise(None)) \
        \
        .withColumn('retorno_pl',
                   when(col('patrimonio_liquido') != 0,
                        col('lucro_liquido') / col('patrimonio_liquido')).otherwise(None)) \
        \
        .withColumn('imobilizacao_pl',
                   when(col('patrimonio_liquido') != 0,
                        col('ativo_permanente') / col('patrimonio_liquido')).otherwise(None)) \
        \
        .withColumn('imobilizacao_recursos_nc',
                   when(col('recursos_nao_correntes') != 0,
                        col('ativo_permanente') / col('recursos_nao_correntes')).otherwise(None))
    
    # Preparar tabela de empresas com campos corretos
    df_empresas_prep = df_empresas \
        .select(
            'id_ecd',
            when(col('empresa_grande_porte') == 'Sim', lit('GRANDE'))
                .otherwise(lit('PEQUENA/MÉDIA')).alias('porte_empresa'),
            coalesce(col('cnae_secao_descricao'), lit('NÃO CLASSIFICADO')).alias('setor_atividade')
        ) \
        .dropDuplicates(['id_ecd'])
    
    # Join com informações de empresa
    df_indicadores = df_indicadores.join(
        df_empresas_prep,
        on='id_ecd',
        how='left'
    )
    
    print(f"✅ Indicadores calculados para {df_indicadores.count():,} registros")
    
    return df_indicadores


# ==============================================================================
# CÉLULA 2: CÁLCULO DE ÍNDICES-PADRÃO (DECIS)
# ==============================================================================

def calcular_indices_padrao_setoriais(df_indicadores):
    """
    Calcula índices-padrão (decis) por setor e porte
    Metodologia Matarazzo (1998)
    """
    
    print("📈 Calculando índices-padrão setoriais (decis)...")
    
    # Lista de indicadores para calcular decis
    indicadores = [
        'liquidez_corrente', 'liquidez_geral', 'endividamento',
        'participacao_capital_proprio', 'giro_ativo', 'margem_liquida',
        'retorno_ativo', 'retorno_pl', 'imobilizacao_pl', 'imobilizacao_recursos_nc'
    ]
    
    # Filtrar apenas registros válidos (sem nulos)
    df_validos = df_indicadores.filter(
        ' AND '.join([f'{ind} IS NOT NULL' for ind in indicadores])
    )
    
    # Calcular decis por setor e porte usando percentile_approx
    decis_list = []
    
    for indicador in indicadores:
        print(f"  Calculando decis para: {indicador}")
        
        df_decis = df_validos.groupBy('setor_atividade', 'porte_empresa').agg(
            expr(f"percentile_approx({indicador}, array(0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9))").alias('decis'),
            count('*').alias('n_empresas'),
            mean(indicador).alias('media'),
            stddev(indicador).alias('desvio_padrao')
        ).withColumn('indicador', lit(indicador))
        
        # Expandir array de decis em colunas
        df_decis = df_decis \
            .withColumn('decil_1', col('decis')[0]) \
            .withColumn('decil_2', col('decis')[1]) \
            .withColumn('decil_3', col('decis')[2]) \
            .withColumn('decil_4', col('decis')[3]) \
            .withColumn('mediana', col('decis')[4]) \
            .withColumn('decil_6', col('decis')[5]) \
            .withColumn('decil_7', col('decis')[6]) \
            .withColumn('decil_8', col('decis')[7]) \
            .withColumn('decil_9', col('decis')[8]) \
            .drop('decis')
        
        decis_list.append(df_decis)
    
    # Consolidar todos os decis
    df_indices_padrao = decis_list[0]
    for df in decis_list[1:]:
        df_indices_padrao = df_indices_padrao.union(df)
    
    print(f"✅ Índices-padrão calculados: {df_indices_padrao.count():,} registros")
    
    return df_indices_padrao


# ==============================================================================
# CÉLULA 3: CLASSIFICAÇÃO POR ÍNDICES-PADRÃO
# ==============================================================================

def classificar_por_indices_padrao(df_indicadores, df_indices_padrao):
    """
    Classifica empresas usando escala qualitativa de Matarazzo:
    PÉSSIMO, DEFICIENTE, FRACO, RAZOÁVEL, SATISFATÓRIO, BOM, ÓTIMO
    """
    
    print("🎯 Classificando empresas por índices-padrão...")
    
    # Função para determinar conceito baseado em decis
    def classificar_decil(valor, decis, quanto_maior_melhor=True):
        """
        Quanto maior melhor: liquidez, rentabilidade
        Quanto menor melhor: endividamento, imobilização
        """
        if quanto_maior_melhor:
            return when(valor >= decis['decil_9'], lit('ÓTIMO')) \
                .when(valor >= decis['decil_8'], lit('BOM')) \
                .when(valor >= decis['decil_6'], lit('SATISFATÓRIO')) \
                .when(valor >= decis['mediana'], lit('RAZOÁVEL')) \
                .when(valor >= decis['decil_3'], lit('FRACO')) \
                .when(valor >= decis['decil_1'], lit('DEFICIENTE')) \
                .otherwise(lit('PÉSSIMO'))
        else:
            return when(valor <= decis['decil_1'], lit('ÓTIMO')) \
                .when(valor <= decis['decil_2'], lit('BOM')) \
                .when(valor <= decis['decil_4'], lit('SATISFATÓRIO')) \
                .when(valor <= decis['mediana'], lit('RAZOÁVEL')) \
                .when(valor <= decis['decil_7'], lit('FRACO')) \
                .when(valor <= decis['decil_9'], lit('DEFICIENTE')) \
                .otherwise(lit('PÉSSIMO'))
    
    # Mapear direção de cada indicador
    direcao_indicadores = {
        # Quanto maior, melhor
        'liquidez_corrente': True,
        'liquidez_geral': True,
        'participacao_capital_proprio': True,
        'giro_ativo': True,
        'margem_liquida': True,
        'retorno_ativo': True,
        'retorno_pl': True,
        # Quanto menor, melhor
        'endividamento': False,
        'imobilizacao_pl': False,
        'imobilizacao_recursos_nc': False
    }
    
    # Join com índices-padrão e classificar
    df_classificado = df_indicadores
    
    for indicador, maior_melhor in direcao_indicadores.items():
        # Join com tabela de decis
        df_decis_ind = df_indices_padrao.filter(col('indicador') == indicador) \
            .select('setor_atividade', 'porte_empresa', 'mediana', 
                   'decil_1', 'decil_2', 'decil_3', 'decil_4', 
                   'decil_6', 'decil_7', 'decil_8', 'decil_9')
        
        df_classificado = df_classificado.join(
            df_decis_ind,
            on=['setor_atividade', 'porte_empresa'],
            how='left'
        )
        
        # Aplicar classificação
        df_classificado = df_classificado.withColumn(
            f'conceito_{indicador}',
            classificar_decil(col(indicador), {
                'mediana': 'mediana',
                'decil_1': 'decil_1', 'decil_2': 'decil_2', 'decil_3': 'decil_3',
                'decil_4': 'decil_4', 'decil_6': 'decil_6', 'decil_7': 'decil_7',
                'decil_8': 'decil_8', 'decil_9': 'decil_9'
            }, maior_melhor)
        )
        
        # Limpar colunas temporárias
        df_classificado = df_classificado.drop('mediana', 'decil_1', 'decil_2', 'decil_3',
                                               'decil_4', 'decil_6', 'decil_7', 'decil_8', 'decil_9')
    
    print(f"✅ Empresas classificadas: {df_classificado.count():,} registros")
    
    return df_classificado


# ==============================================================================
# CÉLULA 4: FATOR DE INSOLVÊNCIA DE KANITZ
# ==============================================================================

def calcular_fator_kanitz(df_indicadores):
    """
    Calcula o Fator de Insolvência de Kanitz
    FI = 0.05*X1 + 1.65*X2 + 3.55*X3 - 1.06*X4 - 0.33*X5
    
    Interpretação:
    - FI > 0: Solvente (baixo risco)
    - -3 < FI <= 0: Penumbra (risco moderado)
    - FI <= -3: Insolvente (alto risco)
    """
    
    print("⚖️ Calculando Fator de Insolvência de Kanitz...")
    
    df_kanitz = df_indicadores \
        .withColumn('x1_roa', col('retorno_ativo')) \
        .withColumn('x2_liquidez_ajustada', 
                   when((col('passivo_circulante') + col('exigivel_longo_prazo')) != 0,
                        (col('ativo_circulante') + col('realizavel_longo_prazo')) / 
                        (col('passivo_circulante') + col('exigivel_longo_prazo'))).otherwise(None)) \
        .withColumn('x3_liquidez_seca',
                   when(col('passivo_circulante') != 0,
                        (col('ativo_circulante') - col('custo_mercadorias')) / col('passivo_circulante')).otherwise(None)) \
        .withColumn('x4_liquidez_corrente', col('liquidez_corrente')) \
        .withColumn('x5_endividamento_pl',
                   when(col('patrimonio_liquido') != 0,
                        (col('passivo_circulante') + col('exigivel_longo_prazo')) / col('patrimonio_liquido')).otherwise(None))
    
    # Calcular Fator de Kanitz
    df_kanitz = df_kanitz.withColumn(
        'fator_kanitz',
        (0.05 * col('x1_roa')) +
        (1.65 * col('x2_liquidez_ajustada')) +
        (3.55 * col('x3_liquidez_seca')) -
        (1.06 * col('x4_liquidez_corrente')) -
        (0.33 * col('x5_endividamento_pl'))
    )
    
    # Classificar situação financeira
    df_kanitz = df_kanitz.withColumn(
        'situacao_kanitz',
        when(col('fator_kanitz') > 0, lit('SOLVENTE'))
        .when(col('fator_kanitz') > -3, lit('PENUMBRA'))
        .otherwise(lit('INSOLVENTE'))
    )
    
    # Adicionar score normalizado (0-100)
    df_kanitz = df_kanitz.withColumn(
        'score_kanitz',
        when(col('fator_kanitz') >= 0, 
             least(lit(100), 50 + (col('fator_kanitz') * 10)))
        .when(col('fator_kanitz') >= -3,
             least(lit(50), greatest(lit(25), 50 + (col('fator_kanitz') * 8.33))))
        .otherwise(
             greatest(lit(0), 25 + ((col('fator_kanitz') + 3) * 8.33)))
    )
    
    print(f"✅ Fator Kanitz calculado: {df_kanitz.count():,} registros")
    
    # Estatísticas
    df_kanitz.groupBy('situacao_kanitz').count().orderBy(desc('count')).show()
    
    return df_kanitz


# ==============================================================================
# CÉLULA 5: CLASSIFICAÇÃO HÍBRIDA (COMBINANDO 3 METODOLOGIAS)
# ==============================================================================

def criar_classificacao_hibrida(df_saldos, df_empresas, df_ml_features):
    """
    Combina 3 metodologias de classificação:
    1. Estrutural (SQL/Regras) - 30%
    2. Índices-Padrão (Matarazzo) - 40%
    3. Features ML (Dicionários Temáticos) - 30%
    
    Retorna score ponderado e classificação final
    """
    
    print("🔄 Criando classificação híbrida...")
    
    # 1. Calcular indicadores financeiros
    df_indicadores = calcular_indicadores_economico_financeiros(df_saldos, df_empresas)
    
    # 2. Calcular índices-padrão setoriais
    df_indices_padrao = calcular_indices_padrao_setoriais(df_indicadores)
    
    # 3. Classificar por índices-padrão
    df_class_padrao = classificar_por_indices_padrao(df_indicadores, df_indices_padrao)
    
    # 4. Calcular Fator Kanitz
    df_kanitz = calcular_fator_kanitz(df_class_padrao)
    
    # 5. Converter conceitos em scores (0-100)
    conceito_to_score = {
        'PÉSSIMO': 10,
        'DEFICIENTE': 25,
        'FRACO': 40,
        'RAZOÁVEL': 55,
        'SATISFATÓRIO': 70,
        'BOM': 85,
        'ÓTIMO': 100
    }
    
    # Criar mapping UDF
    conceito_mapping = create_map([lit(x) for pair in conceito_to_score.items() for x in pair])
    
    # Calcular scores de cada metodologia
    df_hibrido = df_kanitz
    
    # ============================================================================
    # CALCULAR SCORE HÍBRIDO (VERSÃO CORRIGIDA)
    # ============================================================================
    
    # Lista de indicadores
    indicadores_conceito = [
        'conceito_liquidez_corrente', 
        'conceito_liquidez_geral',
        'conceito_endividamento', 
        'conceito_retorno_ativo', 
        'conceito_retorno_pl'
    ]
    
    # Criar scores individuais
    for col_conceito in indicadores_conceito:
        df_hibrido = df_hibrido.withColumn(
            f'score_{col_conceito}',
            conceito_mapping[col(col_conceito)]
        )
    
    # Score Estrutural (média) ✅ LINHA CORRIGIDA
    df_hibrido = df_hibrido.withColumn(
        'score_estrutural',
        (
            coalesce(col('score_conceito_liquidez_corrente'), lit(0)) +
            coalesce(col('score_conceito_liquidez_geral'), lit(0)) +
            coalesce(col('score_conceito_endividamento'), lit(0)) +
            coalesce(col('score_conceito_retorno_ativo'), lit(0)) +
            coalesce(col('score_conceito_retorno_pl'), lit(0))
        ) / 5.0
    )
    
    # Score Índices-Padrão
    df_hibrido = df_hibrido.withColumn('score_indices_padrao', col('score_kanitz'))
    
    # Score ML
    df_hibrido = df_hibrido.join(
        df_ml_features.select('id_ecd', 'dt_referencia', 
                             (col('confianca_media') * 100).alias('score_ml')),
        on=['id_ecd', 'dt_referencia'],
        how='left'
    )
    
    # Score Final Híbrido
    df_hibrido = df_hibrido.withColumn(
        'score_hibrido',
        (coalesce(col('score_estrutural'), lit(0)) * 0.30) +
        (coalesce(col('score_indices_padrao'), lit(0)) * 0.40) +
        (coalesce(col('score_ml'), lit(0)) * 0.30)
    )
    
    # Classificação Final
    df_hibrido = df_hibrido.withColumn(
        'classificacao_hibrida',
        when(col('score_hibrido') >= 85, lit('EXCELENTE'))
        .when(col('score_hibrido') >= 70, lit('BOM'))
        .when(col('score_hibrido') >= 55, lit('REGULAR'))
        .when(col('score_hibrido') >= 40, lit('RUIM'))
        .otherwise(lit('CRÍTICO'))
    )
    
    df_hibrido = df_hibrido.withColumn(
        'nivel_risco',
        when(col('score_hibrido') >= 70, lit('BAIXO'))
        .when(col('score_hibrido') >= 50, lit('MÉDIO'))
        .when(col('score_hibrido') >= 30, lit('ALTO'))
        .otherwise(lit('CRÍTICO'))
    )
    
    print("✅ Classificação híbrida completa!")
    
    # Estatísticas finais
    print("\n📊 Distribuição da Classificação Híbrida:")
    df_hibrido.groupBy('classificacao_hibrida', 'nivel_risco').count().orderBy(desc('count')).show()
    
    return df_hibrido, df_indices_padrao


# ==============================================================================
# CÉLULA 6: SALVAR RESULTADOS EM TABELAS HIVE
# ==============================================================================

def salvar_analise_indices_padrao(spark, df_hibrido, df_indices_padrao):
    """
    Salva todos os resultados em tabelas Hive
    """
    
    print("💾 Salvando resultados em tabelas Hive...")
    
    # 1. Tabela de indicadores e classificação híbrida
    print("  Salvando: teste.ecd_indicadores_hibrido")
    df_hibrido.write.mode('overwrite').saveAsTable('teste.ecd_indicadores_hibrido')
    
    # 2. Tabela de índices-padrão setoriais (decis)
    print("  Salvando: teste.ecd_indices_padrao_setoriais")
    df_indices_padrao.write.mode('overwrite').saveAsTable('teste.ecd_indices_padrao_setoriais')
    
    # 3. Tabela resumo por empresa (última classificação)
    print("  Salvando: teste.ecd_empresas_classificacao_resumo")
    window_ultima = Window.partitionBy('id_ecd').orderBy(desc('dt_referencia'))
    
    df_resumo = df_hibrido \
        .withColumn('rn', row_number().over(window_ultima)) \
        .filter(col('rn') == 1) \
        .select(
            'id_ecd', 'dt_referencia', 'setor_atividade', 'porte_empresa',
            'score_estrutural', 'score_indices_padrao', 'score_ml', 'score_hibrido',
            'classificacao_hibrida', 'nivel_risco', 'situacao_kanitz', 'fator_kanitz',
            'liquidez_corrente', 'liquidez_geral', 'endividamento',
            'retorno_ativo', 'retorno_pl', 'margem_liquida'
        ) \
        .drop('rn')
    
    df_resumo.write.mode('overwrite').saveAsTable('teste.ecd_empresas_classificacao_resumo')
    
    # 4. Tabela de evolução temporal
    print("  Salvando: teste.ecd_evolucao_scores")
    df_evolucao = df_hibrido.select(
        'id_ecd', 'dt_referencia', 
        'score_estrutural', 'score_indices_padrao', 'score_ml', 'score_hibrido',
        'classificacao_hibrida', 'nivel_risco'
    )
    
    df_evolucao.write.mode('overwrite').saveAsTable('teste.ecd_evolucao_scores')
    
    # 5. Tabela de empresas em situação crítica
    print("  Salvando: teste.ecd_empresas_criticas")
    df_criticas = df_hibrido.filter(
        (col('nivel_risco').isin(['ALTO', 'CRÍTICO'])) |
        (col('situacao_kanitz') == 'INSOLVENTE') |
        (col('score_hibrido') < 40)
    ).select(
        'id_ecd', 'dt_referencia', 'setor_atividade', 'porte_empresa',
        'score_hibrido', 'nivel_risco', 'situacao_kanitz', 'fator_kanitz',
        'classificacao_hibrida'
    )
    
    df_criticas.write.mode('overwrite').saveAsTable('teste.ecd_empresas_criticas')
    
    print("\n✅ Todas as tabelas salvas com sucesso!")
    print("\nTabelas disponíveis:")
    print("  1. teste.ecd_indicadores_hibrido - Análise completa")
    print("  2. teste.ecd_indices_padrao_setoriais - Decis setoriais")
    print("  3. teste.ecd_empresas_classificacao_resumo - Resumo por empresa")
    print("  4. teste.ecd_evolucao_scores - Evolução temporal")
    print("  5. teste.ecd_empresas_criticas - Empresas em risco")
    
    return {
        'indicadores_hibrido': df_hibrido,
        'indices_padrao': df_indices_padrao,
        'resumo': df_resumo,
        'evolucao': df_evolucao,
        'criticas': df_criticas
    }



# ==============================================================================
# EXEMPLO DE USO COMPLETO COM CAMPOS CORRETOS
# ==============================================================================


# ============================================================================
# PASSO 1: CARREGAR DADOS COM NOMES CORRETOS
# ============================================================================
df_saldos, df_empresas, df_ml = carregar_dados_ecd_filtrados()
df_empresas = spark.table('teste.ecd_empresas_cadastro')
df_ml_features = spark.table('teste.ecd_ml_empresas_resumo')

print(f"📊 Dados carregados:")
print(f"   - Saldos: {df_saldos.count():,} registros")
print(f"   - Empresas: {df_empresas.count():,} registros")
print(f"   - ML Features: {df_ml_features.count():,} registros")

# ============================================================================
# PASSO 2: CALCULAR INDICADORES (VERSÃO CORRIGIDA)
# ============================================================================
df_indicadores = calcular_indicadores_economico_financeiros(df_saldos, df_empresas)

print(f"✅ Indicadores calculados")
df_indicadores.printSchema()
df_indicadores.select('id_ecd', 'dt_referencia', 'liquidez_corrente', 
                     'retorno_ativo', 'endividamento').show(10)

# ============================================================================
# PASSO 3: CALCULAR ÍNDICES-PADRÃO SETORIAIS
# ============================================================================
df_indices_padrao = calcular_indices_padrao_setoriais(df_indicadores)

print(f"✅ Índices-padrão calculados")
df_indices_padrao.show(10)

# ============================================================================
# PASSO 4: CALCULAR FATOR KANITZ
# ============================================================================
df_kanitz = calcular_fator_kanitz(df_indicadores)

print(f"✅ Fator Kanitz calculado")
df_kanitz.groupBy('situacao_kanitz').count().show()


# ============================================================================
# PASSO 5: CRIAR CLASSIFICAÇÃO HÍBRIDA (VERSÃO CORRIGIDA)
# ============================================================================

print("\n🔄 Criando classificação híbrida...")

# Classificar por índices-padrão
df_class_padrao = classificar_por_indices_padrao(df_indicadores, df_indices_padrao)

# ✅ CORREÇÃO 1: Remover colunas duplicadas ANTES do JOIN
colunas_para_dropar = [
    'ativo_circulante', 'realizavel_longo_prazo', 'ativo_permanente', 'ativo_total',
    'passivo_circulante', 'exigivel_longo_prazo', 'patrimonio_liquido',
    'receita_bruta', 'deducoes_receita', 'receita_liquida', 'lucro_liquido', 'custo_mercadorias',
    'exigivel_total', 'recursos_nao_correntes',
    'liquidez_corrente', 'liquidez_geral', 'endividamento', 'participacao_capital_proprio',
    'giro_ativo', 'margem_liquida', 'retorno_ativo', 'retorno_pl',
    'imobilizacao_pl', 'imobilizacao_recursos_nc',
    'setor_atividade', 'porte_empresa'
]

# Dropar colunas do df_class_padrao que já existem em df_kanitz
df_class_padrao_limpo = df_class_padrao.drop(*[c for c in colunas_para_dropar if c in df_class_padrao.columns])

# Juntar com Kanitz (agora sem duplicatas)
df_hibrido = df_kanitz.join(
    df_class_padrao_limpo,
    on=['id_ecd', 'dt_referencia'],
    how='left'
)

print(f"✅ JOIN realizado sem duplicatas: {len(df_hibrido.columns)} colunas")

# Criar scores híbridos
conceito_to_score = {
    'PÉSSIMO': 10, 'DEFICIENTE': 25, 'FRACO': 40,
    'RAZOÁVEL': 55, 'SATISFATÓRIO': 70, 'BOM': 85, 'ÓTIMO': 100
}
conceito_mapping = create_map([lit(x) for pair in conceito_to_score.items() for x in pair])

# Score estrutural (média dos conceitos)
indicadores_conceito = [
    'conceito_liquidez_corrente', 'conceito_liquidez_geral',
    'conceito_endividamento', 'conceito_retorno_ativo', 'conceito_retorno_pl'
]

for col_conceito in indicadores_conceito:
    df_hibrido = df_hibrido.withColumn(
        f'score_{col_conceito}',
        conceito_mapping[col(col_conceito)]
    )

df_hibrido = df_hibrido.withColumn(
    'score_estrutural',
    (
        coalesce(col('score_conceito_liquidez_corrente'), lit(0)) +
        coalesce(col('score_conceito_liquidez_geral'), lit(0)) +
        coalesce(col('score_conceito_endividamento'), lit(0)) +
        coalesce(col('score_conceito_retorno_ativo'), lit(0)) +
        coalesce(col('score_conceito_retorno_pl'), lit(0))
    ) / 5.0
)

# Score índices-padrão (Kanitz normalizado)
df_hibrido = df_hibrido.withColumn('score_indices_padrao', col('score_kanitz'))

# Score ML
df_hibrido = df_hibrido.join(
    df_ml_features.select('id_ecd', 'dt_referencia', 
                         (col('confianca_media') * 100).alias('score_ml')),
    on=['id_ecd', 'dt_referencia'],
    how='left'
)

# Score Final Híbrido
df_hibrido = df_hibrido.withColumn(
    'score_hibrido',
    (coalesce(col('score_estrutural'), lit(0)) * 0.30) +
    (coalesce(col('score_indices_padrao'), lit(0)) * 0.40) +
    (coalesce(col('score_ml'), lit(0)) * 0.30)
)

# Classificação final
df_hibrido = df_hibrido.withColumn(
    'classificacao_hibrida',
    when(col('score_hibrido') >= 85, lit('EXCELENTE'))
    .when(col('score_hibrido') >= 70, lit('BOM'))
    .when(col('score_hibrido') >= 55, lit('REGULAR'))
    .when(col('score_hibrido') >= 40, lit('RUIM'))
    .otherwise(lit('CRÍTICO'))
)

df_hibrido = df_hibrido.withColumn(
    'nivel_risco',
    when(col('score_hibrido') >= 70, lit('BAIXO'))
    .when(col('score_hibrido') >= 50, lit('MÉDIO'))
    .when(col('score_hibrido') >= 30, lit('ALTO'))
    .otherwise(lit('CRÍTICO'))
)

print("✅ Classificação híbrida completa!")

# Estatísticas
df_hibrido.groupBy('classificacao_hibrida', 'nivel_risco').count().orderBy(desc('count')).show()

# ============================================================================
# SALVAR TABELAS (VERSÃO FINAL CORRIGIDA)
# ============================================================================

print("\n💾 Salvando tabelas...")

# ✅ CORREÇÃO 2: Verificar se há duplicatas
colunas_originais = df_hibrido.columns
colunas_unicas = list(dict.fromkeys(colunas_originais))  # Remove duplicatas mantendo ordem

if len(colunas_originais) != len(colunas_unicas):
    print(f"  ⚠️ Encontradas {len(colunas_originais) - len(colunas_unicas)} colunas duplicadas")
    duplicadas = [c for c in colunas_unicas if colunas_originais.count(c) > 1]
    print(f"  Colunas duplicadas: {duplicadas}")
    
    # Reconstruir DataFrame sem duplicatas
    df_hibrido = df_hibrido.toDF(*[f"{c}_{i}" if colunas_originais[:idx].count(c) > 0 else c 
                                    for idx, c in enumerate(colunas_originais) 
                                    for i in [colunas_originais[:idx].count(c)]])
    
    # Selecionar apenas primeiras ocorrências
    df_hibrido = df_hibrido.select(colunas_unicas)

print(f"  ✅ DataFrame final: {len(df_hibrido.columns)} colunas")

# 1. Indicadores completos
print("  Salvando: teste.ecd_indicadores_hibrido")
df_hibrido.write.mode('overwrite').saveAsTable('teste.ecd_indicadores_hibrido')

# 2. Índices-padrão setoriais
print("  Salvando: teste.ecd_indices_padrao_setoriais")
df_indices_padrao.write.mode('overwrite').saveAsTable('teste.ecd_indices_padrao_setoriais')

# 3. Resumo por empresa (última data)
print("  Salvando: teste.ecd_empresas_classificacao_resumo")
window_ultima = Window.partitionBy('id_ecd').orderBy(desc('dt_referencia'))
df_resumo = df_hibrido \
    .withColumn('rn', row_number().over(window_ultima)) \
    .filter(col('rn') == 1) \
    .drop('rn')

# Selecionar colunas importantes
colunas_resumo = [
    'id_ecd', 'dt_referencia', 'setor_atividade', 'porte_empresa',
    'score_estrutural', 'score_indices_padrao', 'score_ml', 'score_hibrido',
    'classificacao_hibrida', 'nivel_risco', 'situacao_kanitz', 'fator_kanitz',
    'liquidez_corrente', 'liquidez_geral', 'endividamento',
    'retorno_ativo', 'retorno_pl', 'margem_liquida'
]
colunas_resumo_existentes = [c for c in colunas_resumo if c in df_hibrido.columns]
df_resumo.select(colunas_resumo_existentes).write.mode('overwrite').saveAsTable('teste.ecd_empresas_classificacao_resumo')

# 4. Evolução temporal
print("  Salvando: teste.ecd_evolucao_scores")
colunas_evolucao = [
    'id_ecd', 'dt_referencia',
    'score_estrutural', 'score_indices_padrao', 'score_ml', 'score_hibrido',
    'classificacao_hibrida', 'nivel_risco'
]
colunas_evolucao_existentes = [c for c in colunas_evolucao if c in df_hibrido.columns]
df_hibrido.select(colunas_evolucao_existentes).write.mode('overwrite').saveAsTable('teste.ecd_evolucao_scores')

# 5. Empresas críticas
print("  Salvando: teste.ecd_empresas_criticas")
df_criticas = df_hibrido.filter(
    (col('nivel_risco').isin(['ALTO', 'CRÍTICO'])) |
    (col('situacao_kanitz') == 'INSOLVENTE') |
    (col('score_hibrido') < 40)
)
colunas_criticas = [
    'id_ecd', 'dt_referencia', 'setor_atividade', 'porte_empresa',
    'score_hibrido', 'nivel_risco', 'situacao_kanitz', 'fator_kanitz',
    'classificacao_hibrida'
]
colunas_criticas_existentes = [c for c in colunas_criticas if c in df_criticas.columns]
df_criticas.select(colunas_criticas_existentes).write.mode('overwrite').saveAsTable('teste.ecd_empresas_criticas')

print("\n✅ Todas as tabelas salvas com sucesso!")
print("\n📋 Tabelas criadas:")
print("  1. teste.ecd_indicadores_hibrido")
print("  2. teste.ecd_indices_padrao_setoriais")
print("  3. teste.ecd_empresas_classificacao_resumo")
print("  4. teste.ecd_evolucao_scores")
print("  5. teste.ecd_empresas_criticas")

# Estatísticas finais
print("\n📊 ESTATÍSTICAS FINAIS:")
print(f"\nEmpresas Críticas: {spark.table('teste.ecd_empresas_criticas').count():,}")

print("\nTop 10 Melhores:")
spark.table('teste.ecd_empresas_classificacao_resumo') \
    .select('id_ecd', 'score_hibrido', 'classificacao_hibrida', 'nivel_risco') \
    .orderBy(desc('score_hibrido')).show(10)

print("\n✅ ANÁLISE HÍBRIDA CONCLUÍDA!")


# ============================================================================
# PASSO 7: VALIDAÇÕES
# ============================================================================
print("\n📊 ESTATÍSTICAS FINAIS:")

print("\nDistribuição de Classificação:")
spark.table('teste.ecd_empresas_classificacao_resumo') \
    .groupBy('classificacao_hibrida', 'nivel_risco').count() \
    .orderBy(desc('count')).show()

print("\nEmpresas Críticas:")
print(f"Total: {spark.table('teste.ecd_empresas_criticas').count():,}")

print("\nTop 10 Melhores Empresas:")
spark.table('teste.ecd_empresas_classificacao_resumo') \
    .select('id_ecd', 'setor_atividade', 'score_hibrido', 
            'classificacao_hibrida', 'nivel_risco') \
    .orderBy(desc('score_hibrido')) \
    .show(10, truncate=False)

print("\n✅ ANÁLISE HÍBRIDA CONCLUÍDA COM SUCESSO!")


🔄 Carregando dados do ECD com filtros aplicados...

📂 Carregando: teste.ecd_saldos_contas_v2
🔍 APLICANDO FILTROS DE QUALIDADE - SALDOS


📊 Registros antes do filtro: 945,499,468


📊 Registros depois do filtro: 929,622,972
🗑️  Registros removidos (zeros/nulos): 15,876,496


✅ Taxa de retenção: 98.32%

📋 Filtrando contas inválidas...


📊 Registros antes: 929,622,972


...

  Salvando: teste.ecd_evolucao_scores


                                                                                ]]

  Salvando: teste.ecd_empresas_criticas


                                                                                ]]


✅ Todas as tabelas salvas com sucesso!

📋 Tabelas criadas:
  1. teste.ecd_indicadores_hibrido
  2. teste.ecd_indices_padrao_setoriais
  3. teste.ecd_empresas_classificacao_resumo
  4. teste.ecd_evolucao_scores
  5. teste.ecd_empresas_criticas

📊 ESTATÍSTICAS FINAIS:

Empresas Críticas: 1,329,709

Top 10 Melhores:
+------+-------------+---------------------+-----------+
|id_ecd|score_hibrido|classificacao_hibrida|nivel_risco|
+------+-------------+---------------------+-----------+
| 68888|         67.0|              REGULAR|      MÉDIO|
|259576|         67.0|              REGULAR|      MÉDIO|
|259888|         67.0|              REGULAR|      MÉDIO|
| 45940|         67.0|              REGULAR|      MÉDIO|
|478648|         67.0|              REGULAR|      MÉDIO|
|277486|         67.0|              REGULAR|      MÉDIO|
|429942|         67.0|              REGULAR|      MÉDIO|
|  4586|         67.0|              REGULAR|      MÉDIO|
|426493|         67.0|              REGULAR|      MÉDIO|
